# Capstone: SageMaker Studio와 Amazon SageMaker Python SDK를 사용하여 엔드 투 엔드 테이블 형식 데이터 기계 학습 프로젝트 빌드

## 환경 설정

환경 설정을 쉽게 시작할 수 있도록 기본 설정 코드가 제공됩니다. 이러한 셀을 먼저 확인한 다음 실행하여 패키지를 설치하고 변수를 생성합니다.

In [ ]:
%%capture
%pip install -U shap
%pip install -U smdebug
%pip install imbalanced-learn
%pip install pytest-cov
%pip install pytest-filter-subpackage
%pip install sagemaker
%pip install -U seaborn

In [ ]:
#required-libraries

import boto3
import datetime as datetime
import io
import IPython
import json
import math
import matplotlib.pyplot as plt  # visualization
import numpy as np
import pandas as pd
import pathlib
import re
import sagemaker
import seaborn as sns  # visualization
import statistics
import string
import sys
import time
import zipfile

from imblearn.over_sampling import SMOTE

from sagemaker import clarify
from sagemaker import get_execution_role
from sagemaker.analytics import ExperimentAnalytics
from sagemaker.dataset_definition.inputs import AthenaDatasetDefinition, DatasetDefinition, RedshiftDatasetDefinition
from sagemaker.debugger import CollectionConfig, DebuggerHookConfig, FrameworkProfile, ProfilerConfig, ProfilerRule, Rule, rule_configs
from sagemaker.estimator import Estimator
from sagemaker.experiments.run import Run, load_run
from sagemaker.feature_store.feature_definition import FeatureDefinition
from sagemaker.feature_store.feature_definition import FeatureTypeEnum
from sagemaker.feature_store.feature_group import FeatureGroup
from sagemaker.inputs import CreateModelInput
from sagemaker.inputs import TrainingInput
from sagemaker.inputs import TransformInput
from sagemaker.model import Model
from sagemaker.model_metrics import MetricsSource, ModelMetrics
from sagemaker.network import NetworkConfig
from sagemaker.processing import FeatureStoreOutput
from sagemaker.processing import Processor, ProcessingInput, ProcessingOutput, ScriptProcessor
from sagemaker.pytorch.estimator import PyTorch
from sagemaker.s3 import S3Uploader
from sagemaker.session import Session
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.transformer import Transformer
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner
from sagemaker.workflow.condition_step import ConditionStep, JsonGet
from sagemaker.workflow.conditions import ConditionGreaterThan
from sagemaker.workflow.parameters import ParameterInteger, ParameterFloat, ParameterString
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.properties import PropertyFile
from sagemaker.workflow.step_collections import RegisterModel
from sagemaker.workflow.steps import CreateModelStep
from sagemaker.workflow.steps import ProcessingStep, TrainingStep
from sagemaker.workflow.steps import TransformStep
from sagemaker.workflow.steps import TuningStep
from sagemaker.xgboost.estimator import XGBoost
from sklearn.model_selection import train_test_split

In [ ]:
#basic-variable-code-and-settings

%matplotlib inline
base_job_name = "capstone-smdebugger-job"
bucket = sagemaker.Session().default_bucket()
bucket_path = "s3://{}".format(bucket)
prefix = "sagemaker/capstone"
region = boto3.Session().region_name
role = sagemaker.get_execution_role()
s3_client = boto3.client("s3")
sagemaker_session = sagemaker.Session()
save_interval = 5
sns.set(color_codes=True)

## 데이터 집합 설명

Amazon Simple Storage Service(Amazon S3) 버킷에는 다음의 5개 테이블이 저장되어 있습니다.
- **claims.csv**: 원시 보험금 청구 데이터가 포함된 테이블
- **customers.csv**: 원시 고객 데이터가 포함된 테이블
- **claims_preprocessed.csv**: 처리된 보험금 청구 데이터가 포함된 테이블
- **customers_preprocessed.csv**: 처리된 고객 데이터가 포함된 테이블
- **claims_customer.csv**: 처리된 보험금 청구 및 고객 데이터의 **policy_id**에 조인된 테이블

이 실습에서는 **claims.csv** 및 **customers.csv** 테이블부터 시작합니다. 구체적으로는 **도전 과제 1**에서 Amazon SageMaker Data Wrangler를 사용하여 이러한 테이블을 처리합니다. 문제가 발생하거나 처리된 데이터 집합에 대한 참조가 필요한 경우, 사전 처리된 테이블을 검토할 수 있습니다.

이 데이터 집합의 대상은 claims 테이블의 **fraud** 열입니다.

claim 테이블에는 다음 필드가 포함되어 있습니다. 

- **policy_id**: 보험의 고유 ID
- **driver_relationship**: 관계 목록(Spouse, Self, Child, Other, N/A)
- **incident_type**: 보고된 사건 유형(Break-In, Collision, Theft)
- **collision_type**: 충돌 위치(Front, Rear, Side, N/A)
- **incident_severity**: 사건의 심각도(Minor, Major, Totaled)
- **authorities_contacted**: 처음 연락한 기관의 유형(None, Police, Ambulance, Fire)
- **num_vehicles_involved**: 사건와 연관된 차량 수(1~6 사이의 수)
- **num_injuries**: 사건에서 발생한 부상자 수(1~4 사이의 수)
- **num_witnesses**: 사건의 목격자 수(1~5 사이의 수)
- **police_report_available**: 경찰 신고서 사용 가능 여부(yes 또는 no)
- **injury_claim**: 청구한 대인 배상 금액(미화 300~576,300달러)
- **vehicle_claim**: 차량 손상과 관련하여 청구한 금액(미화 1,000~51,051달러)
- **total_claim_amount**: 청구한 총 대인/대물 배상 금액(미화 2,100~588,868달러)
- **incident_month**: 사건 발생 월(1~12 사이의 수)
- **incident_day**: 사건 발생 날짜(1~31 사이의 수)
- **incident_dow**: 사건 발생 요일(일요일~토요일에 해당하는 0~6 사이의 수)
- **incident_hour**: 사건 발생 시간(0~23 사이의 수)
- **fraud**: 보험 사기 여부(0 또는 1)

customers 테이블에는 다음 필드가 포함되어 있습니다.

- **policy_id**: 보험의 고유 ID
- **customer_age**: 고객의 연령(18~70 사이의 수)
- **months_as_customer**: 고객의 보험료 납부 기간(월 단위, 1~495 사이의 수)
- **num_claims_past_year**: 지난 해에 고객이 청구한 보험금 액수
- **num_insurers_past_5_years**: 지난 5년 동안 고객이 가입한 보험 수
- **policy_state**: 고객의 거주지 관할 주(AZ, CA, ID, NV, OR, WA)
- **policy_deductable**: 보험 공제 가능액(미화 750~1,100달러)
- **policy_annual_premium**: 보험의 연간 할증료(미화 2,200~3,000달러)
- **policy_liability**: 최대 대인 배상 책임 금액(개별 대인 배상과 전체 대인 배상으로 구분 - 15/30, 25/50, 60/90, 100/200)
- **customer_zip**: 고객 거주지의 우편 번호(83201~99362 사이의 수)
- **customer_gender**: 고객의 성별(Male, Female, Other, Unknown)
- **customer_education**: 고객의 학력(Below High School, High School, Associate, Bachelor, Advanced Degree)
- **auto_year**: 차량 제조 연도(2001~2020 사이의 수)

**policy_id** 열에서 내부 조인을 통해 이러한 테이블을 조인할 수 있습니다.

## 도전 과제 실습 살펴보기

이 실습에 설정되어 있는 링크를 클릭하면 도전 과제와 노트북 끝부분의 색인 간에 이동할 수 있습니다. 색인의 항목을 검토하려는 경우 그와 연관된 하이퍼링크를 선택합니다. 현재 진행 중인 도전 과제로 돌아가려면 색인에서 해당 과제 하이퍼링크를 선택합니다.

이 실습은 다음과 같이 구성되어 있습니다.

- 도전 과제 1: SageMaker Data Wrangler를 사용하여 데이터 집합 분석 및 준비
- 도전 과제 2: SageMaker Feature Store에서 특성 그룹 생성
- 도전 과제 3: 모델 훈련하기
- 도전 과제 4: 모델을 평가하여 편향 유무 확인
- 도전 과제 5: 배치 변환
- 도전 과제 6: 자동 파이프라인 빌드
- 부록

## 도전 과제 1: SageMaker Data Wrangler를 사용하여 데이터 집합 분석 및 준비

AnyCompany Consulting은 자동차 보험 사기 데이터 집합을 분석한 후 신규 보험금 청구 건이 사기일 가능성을 예측하는 데 사용 가능한 모델을 빌드해 달라는 요청을 받았습니다. AnyCompany Consulting이 보유 중인 고객 레코드 5천 개의 각 보험금 청구 건에는 사기 여부 레이블이 적용되어 있습니다. 이 데이터를 사용하여 모델을 훈련하고 검정 및 검증한 후 신규 레코드 배치 컬렉션에서 추론을 실행할 수 있습니다.

그리고 Amazon SageMaker Data Wrangler 분석 기능을 사용하여 중요한 열의 데이터 분산을 시각화하고 열 간의 상관 관계와 대상 유출을 확인합니다. 그 후에는 간단한 기준 모델을 빌드합니다. 그런 다음, SageMaker Data Wrangler의 데이터 처리 기능을 사용해 성능이 더 우수한 모델을 훈련하는 데 더욱 적합하도록 열을 변환합니다. 

이 과제를 완료하려면 다음 하위 과제를 완료해야 합니다.

- 데이터 검토
- Amazon SageMaker Studio에서 정보 탐색을 위한 데이터 분석 완료
- Amazon SageMaker Clarify 프로세서 작업을 사용하여 편향 보고서 실행
- 데이터 준비

이 문제는 완료하는 데 약 *100*분이 소요됩니다.

### 과제 1.1: 데이터 검토

<a id="task1-1-continue"></a>

리포지토리에 저장되어 있는 테이블 형식 자동차 보험 데이터 집합에 액세스하여 데이터 집합 표본을 검토합니다. 리포지토리에는 처리되지 않은 두 개의 테이블이 있습니다. 하나는 **customers.csv**라는 고객 데이터용 테이블이고 다른 하나는 **claims.csv**라는 보험금 청구 데이터용 테이블입니다.

**힌트 1**: 미처리 테이블은 **./data/** 폴더에 있습니다.

**힌트 2**: **claims.csv** 및 **customers.csv** 테이블은 미처리 테이블입니다.

테이블을 잠시 살펴보면서 눈에 띄는 필드가 있는지 그리고 전처리 시 주의해야 하는 필드가 있는지 확인합니다.

데이터를 검토하는 방법에 관한 자세한 단계별 지침은 *부록* 섹션의 <a href="#task1-1" target="_self">**데이터 검토(과제 1.1)**</a>를 참조하세요.

자동차 보험 사기 테이블에 액세스하여 데이터 집합 샘플을 검토했다면 이 과제를 완료한 것입니다.

In [ ]:
#add_your_task_1_1_code_here



### 과제 1.2: SageMaker Studio에서 정보 탐색을 위한 데이터 분석 완료 

<a id="task1-2-continue"></a>

정보 탐색을 위한 데이터 분석을 완료하려면 데이터를 검토하고 데이터 집합에 포함될 수 있는 문제를 파악한 후 데이터 집합의 열과 대상 간에 명확한 상관 관계가 있는지 확인합니다. SageMaker Data Wrangler와 노트북에서 데이터를 탐색할 수 있습니다.

구체적으로는 다음 항목을 검토해야 합니다.
- **열 히스토그램**: 시각적 형식으로 열을 검토하여 데이터 집합에 포함된 값의 종류를 확인합니다.
- **빠른 모델**: 데이터 집합을 검토하여 완성할 수 있는 모델을 예상해 봅니다.
- **특성 상관 관계**: 데이터 집합의 열과 대상 간에 명확한 상관 관계가 설정되어 있는지 확인합니다.
- **대상 유출**: 대상 값에 따라 달라지는 데이터가 있는지 확인합니다.

SageMaker Data Wrangler를 열면 SageMake Studio에서 새 탭이 열립니다. 여기서 설명하는 지침에 따라 작업을 진행하려면 다음 옵션 중 하나를 사용하세요.
- **옵션 1**: 탭을 나란히 표시합니다. 주 SageMaker Studio 창에서 분할 화면 보기를 생성하려면 **capstone_ko_kr.ipynb** 탭을 옆쪽으로 끌거나 **capstone_ko_kr.ipynb** 탭을 마우스 오른쪽 버튼으로 클릭하여 선택한 후 **New View for Notebook**을 선택합니다. 그러면 SageMaker Data Wrangler 흐름을 사용할 때 지침을 확인할 수 있습니다.
- **옵션 2**: SageMaker Studio 탭을 서로 전환하면서 지침에 따라 작업을 진행합니다. SageMaker Data Wrangler 단계 탐색을 완료한 후 **capstone_ko_kr.ipynb** 탭을 선택하여 노트북으로 돌아옵니다.

새 흐름 파일을 생성할 때 **"The following instance type is not available: ml.m5.4xlarge. Try selecting a different instance below."** 오류가 표시되면 다른 인스턴스 유형을 선택할 수 있습니다. 그런 다음 **ml.m5.8xlarge**를 선택해 보세요.

**An error occurred loading this view** 메시지가 표시되면 **untitled.flow** 탭을 닫고 파일 브라우저에서 흐름 파일을 다시 엽니다.

**힌트 1**: 여러 가지 방식으로 데이터 집합을 살펴볼 수 있습니다. 데이터 집합 살펴보기를 시작하려면 SageMaker Data Wrangler 흐름을 엽니다. 이름에 **databucket-** 이 포함된 S3 버킷에서 **claims.csv**와 **customers.csv**를 모두 SageMaker Data Wrangler로 가져와야 합니다.

**힌트 2**: 두 번째 테이블을 가져오려면 **Data flow**로 돌아온 다음, **Import** 탭을 선택하여 다른 데이터 집합을 가져옵니다.

**힌트 3**: SageMaker Data Wrangler에서는 **Get data insights** 및 **Add analysis**의 두 가지 방식으로 데이터를 탐색할 수 있습니다. 데이터에서 샘플 차트 몇 개를 검토한 다음, 원한다면 노트북에서 다른 플로팅 도구를 사용하여 데이터를 분석할 수 있습니다. 데이터 분석에 사용할 수 있도록 **plt** 및 **sns** 라이브러리가 설치되어 있습니다. 사용법을 잘 알고 있는 분석 도구를 자유롭게 사용하여 데이터 집합을 살펴보면 됩니다.  

**힌트 4**: **policy_id**에서 **Join**을 사용하여 테이블 2개를 조인해 봅니다. 그 후에 다른 인사이트 보고서를 실행합니다. 이러한 테이블에는 **Inner** 조인을 사용할 수 있습니다.

조인된 데이터 집합에서 조인 전보다 더 중요한 결과가 확인되었나요?

SageMaker Studio에서 데이터 집합을 탐색하는 방법에 관한 자세한 단계별 지침은 *부록* 섹션의 <a href="#task1-2-1" target="_self">**SageMaker Studio에서 데이터 집합 탐색(과제 1.2)**</a>을 참조하세요.

노트북에서 데이터 집합을 탐색하는 방법에 관한 자세한 단계별 지침은 *부록* 섹션의 <a href="#task1-2-2" target="_self">**노트북에서 데이터 집합 탐색(과제 1.2)**</a>을 참조하세요.

SageMaker Data Wrangler를 사용하여 데이터를 처리하고 데이터 집합을 탐색했으며 수행하려는 처리 단계를 파악했다면 이 과제를 완료한 것입니다.

In [ ]:
#add_your_task_1_2_code_here



### 과제 1.3: SageMaker Clarify 프로세서 작업을 사용하여 편향 보고서 실행

<a id="task1-3-continue"></a>

SageMaker Clarify를 사용하여 훈련 전 편향 보고서를 실행해 데이터에서 클래스 불균형 현상을 파악합니다. SageMaker Data Wrangler 흐름을 사용하여 SageMaker Studio에서 편향 보고서를 실행합니다.
1. 먼저 두 테이블을 조인하는 것부터 시작합니다.

- 조인: **policy_id**에서 **claims.csv**를 **customers.csv**에 조인하는 **Inner** 조인

**힌트 1**: 훈련 전 편향 보고서를 생성하려면 SageMaker Data Wrangler 흐름에 새 분석을 추가한 후 **Analysis type**에서 **Bias Report**를 선택합니다.

**힌트 2**: 매번 분석할 특성을 다르게 선택하여 편향 보고서를 여러 번 실행할 수 있습니다.

SageMaker Data Wrangler로 테이블을 조인하는 방법에 관한 자세한 단계별 지침은 *부록* 섹션의 <a href="#task1-3-1" target="_self">**SageMaker Studio에서 테이블 조인(과제 1.3)**</a>을 참조하세요.

훈련 전 편향 보고서를 실행하는 방법에 관한 자세한 단계별 지침은 *부록* 섹션의 <a href="#task1-3-2" target="_self">**훈련 전 편향 보고서 실행(과제 1.3)**</a>을 참조하세요.

훈련 전 편향 보고서를 실행하고 확인했다면 이 과제를 완료한 것입니다.

### 과제 1.4: 데이터 준비

<a id="task1-4-continue"></a>

SageMaker Data Wrangler를 사용하여 데이터 집합을 준비합니다. 기본적으로는 다음 변환을 포함하고 원하는 경우 다른 변환도 자유롭게 포함할 수 있습니다.

- 범주형 인코딩(원-핫 인코딩): **authorities_contacted**, **collision_type**, **customer_gender**, **driver_relationship**, **incident_type**, **policy_state**
- 범주형 인코딩(서수형 인코딩): **customer_education**, **policy_liability**, **incident_severity**, **police_report_available**
- 열 구문 분석 유형: **vehicle_claim** 및 **total_claim_amount**를 **Float**에서 **Long**으로 구문 분석
- 열 관리(열 삭제): **customer_zip**.
- 열 관리(열 이동): **fraud**(**Move to start** 사용).
- 열 관리(열 이름 바꾸기): **collision_type_N/A** 및 **driver_relationship_N/A**에서 **_** 기호를 사용해 **/** 기호 제거
- 열 관리(열 이름 바꾸기): **policy_id_0**의 이름을 **policy_id**로 바꾸기

**힌트 1**: SageMaker Data Wrangler 조인을 사용하여 **claims** 테이블을 **customers** 테이블에 조인합니다. 

**힌트 2**: **policy_id** 열에서 두 테이블을 조인합니다.

**힌트 3**: **Add transform** 옵션을 사용하여 변환을 추가합니다.

모델 훈련에 가장 큰 영향을 주는 것으로 생각되는 변환은 어느 것인가요?

SageMaker Data Wrangler를 사용하여 데이터를 준비하는 방법에 관한 자세한 단계별 지침은 *부록* 섹션의 <a href="#task1-4-1" target="_self">**SageMaker Data Wrangler를 사용하여 데이터 준비(과제 1.4)**</a>를 참조하세요.

처리된 데이터의 예제 집합을 가져오려는 경우에는 <a href="#task1-4-2" target="_self">**처리된 데이터의 예제 집합 가져오기**</a>를 참조하세요. 


## 도전 과제 2: SageMaker Feature Store에서 특성 그룹 생성

데이터 집합을 처리했으므로 이번에는 이후 분석에 사용할 특성 및 특성 그룹을 생성합니다. SageMaker Feature Store를 사용하여 이러한 특성 및 특성 그룹을 하나의 특성 그룹에 저장하고 모델 훈련 시 쿼리합니다.

이 과제를 완료하려면 다음 하위 과제를 모두 완료해야 합니다.

1. SageMaker Feature Store로 특성 내보내기
2. Amazon Athena를 사용하여 오프라인 저장소에서 특성 그룹 쿼리

이 문제는 완료하는 데 약 *30*분이 소요됩니다.

### 과제 2.1: SageMaker Feature Store로 특성 내보내기

<a id="task2-1-continue"></a>

SageMaker Data Wrangler **Export to** 기능을 사용하여 사용자 지정 Jupyter Notebook을 생성합니다. 노트북은 특성 정의 및 특성 그룹을 생성합니다. 노트북은 레코드를 특성 그룹에 수집합니다. 노트북에서 다음 단계를 완료합니다.

- record 및 event_time의 값 설정
- 노트북 셀을 실행하여 특성 그룹 생성
- 노트북 셀을 실행하여 생성된 특성 그룹 확인
- 노트북 셀을 실행하여 특성 그룹에 레코드 수집

**힌트 1**: SageMaker Studio에서 이러한 단계를 모두 완료할 수 있습니다. 특성 그룹 생성을 완료한 후 이 노트북으로 돌아와 과제 2.2를 계속 진행할 수 있습니다.

**힌트 2**: 사용자 지정 변환을 추가하여 **event_time** 열을 생성합니다. 

**힌트 3**: 사용자 지정 변환을 추가하는 코드는 다음과 같습니다.
    


In [ ]:
import time
import datetime
import pandas as pd
from pyspark.sql.functions import lit
date_time = datetime.date.today()

df = df.withColumn("event_time", lit(time.mktime(date_time.timetuple())))


**힌트 4**: SageMaker Data Wrangler 흐름 끝부분에서 **+** 아이콘을 선택하고 **Export to** 옵션을 선택한 후 **SageMaker Feature Store (via JupyterNotebook)** 를 선택합니다.

**힌트 5**: **enable_online_store** 값을 **True**에서 **False**로 변경하면 온라인 저장소를 끌 수 있습니다.

추론을 사용하는 대신 SageMaker Feature Store에서 어떤 방식으로 훈련에 사용할 레코드를 저장 및 쿼리할 수 있었나요?

**Export to** 옵션을 사용하여 특성 그룹을 만드는 방법에 관한 자세한 단계별 지침은 *부록* 섹션의 <a href="#task2-1" target="_self">**Export to 옵션을 사용하여 특성 그룹 생성(과제 2.1)**</a>을 참조하세요. 

특성 그룹을 생성하고 해당 특성 그룹에 데이터를 수집했다면 이 과제를 완료한 것입니다.

### 과제 2.2: Athena를 사용하여 오프라인 저장소에서 특성 그룹 쿼리

<a id="task2-2-continue"></a>

Athena를 사용하여 오프라인 데이터 스토어에서 레코드를 추출합니다. 다음 도전 과제에서 이러한 레코드를 훈련, 검정 및 검증 집합으로 분할합니다.

아래에 제공된 코드 셀을 사용하여 Amazon Athena API를 호출합니다. Amazon Athena 콘솔을 사용하여 쿼리를 만들 수 있지만 이 실습의 범위를 벗어납니다.

**힌트 1**: **feature_group.athena_query()** 를 사용하여 Athena 쿼리를 생성하고 **query.table_name**을 사용하여 테이블 이름을 가져올 수 있습니다.

**힌트 2**: **query.run(query_string=query_string, output_location=output_location)** 을 사용하여 쿼리를 실행하고 **query.as_dataframe()** 을 사용하여 반환된 값을 데이터 프레임으로 읽을 수 있습니다.

**event_time**을 사용하여 데이터 집합 타임라인의 여러 지점에서 확인되는 특성을 어떻게 추적할 수 있었나요?

Athena를 사용하여 오프라인 데이터 저장소에서 레코드를 추출하는 방법에 관한 자세한 단계별 지침은 *부록* 섹션의 <a href="#task2-2" target="_self">**Athena를 사용하여 오프라인 저장소에서 레코드 추출(과제 2.2)**</a>을 참조하세요.

반환된 Athena 쿼리를 데이터 프레임 변수로 저장했다면 이 과제를 완료한 것입니다.

In [ ]:
#add_your_task_2_2_code_here



## 도전 과제 3: 모델 훈련하기

이제 모델을 훈련할 수 있습니다. 데이터를 훈련, 검정 및 검증 데이터 집합으로 분할한 후 모델을 훈련합니다. 

이 데이터에서 SageMaker Autopilot을 미리 실행한 결과 **F1**은 **0.616**, **accuracy**는 **0.978**, **AUC**는 **0.918**, **Recall**은 **0.539**였습니다. SageMaker Autopilot이 생성하는 지표에 관한 자세한 내용은 *추가 리소스* 섹션의 *autopilot-metrics-validation* 문서를 참조하세요.

훈련과 튜닝을 진행하면서 SageMaker Autopilot 점수보다 높은 점수를 기록할 때 Amazon SageMaker Debugger에서 오류를 보고하지 않음을 확인합니다.

이 과제를 완료하려면 다음 하위 과제를 완료해야 합니다.

- 실험과 시험 생성
- 데이터를 훈련, 검정 및 검증 데이터 집합으로 분할
- 훈련 작업 구성 및 실행
    - 기본적인 훈련 작업 실행
    - SageMaker Debugger를 사용하도록 설정하여 훈련 작업을 실행한 후 보고서 분석(선택 사항)
- 하이퍼파라미터 튜닝 수행

이 문제는 완료하는 데 약 *110*분이 소요됩니다.


### 과제 3.1: 실험 및 시험 이름 지정

<a id="task3-1-continue"></a>

실험과 시험에 모두 이름을 지정하도록 변수를 설정합니다. 
실험에는 **experiment_name**, **run_name**과 **description**이 필요합니다. 

변수를 만드는 방법에 관한 자세한 단계별 지침은 *부록* 섹션의 <a href="#task3-1" target="_self">**실험 및 시행 이름 지정(과제 3.1)**</a>을 참조하세요.


In [ ]:
#add_your_task_3_1_code_here



### 과제 3.2: 데이터를 훈련, 검정 및 검증 데이터 집합으로 분할

<a id="task3-2-continue"></a>

SageMaker Feature Store에서 쿼리한 특성을 사용하여 데이터를 훈련, 검정 및 검증 데이터 집합으로 분할합니다.

**힌트 1**: **np.split**을 사용하여 데이터 집합을 3개의 파티션으로 분할합니다.

**힌트 2**: **to_csv**를 사용하여 CSV 파일을 생성한 후 **S3Uploader.upload**를 사용하여 Amazon S3에 파일을 추가합니다.

**힌트 3**: 분할에서는 최종적으로 **train** 및 **validation**의 값이 포함된 **data_inputs** 변수가 생성되어야 합니다.

데이터를 훈련, 검정 및 검증 데이터 집합으로 분할하는 방법에 관한 자세한 단계별 지침은 *부록* 섹션의 <a href="#task3-2" target="_self">**데이터를 훈련, 검정 및 검증 데이터 집합으로 분할(과제 3.2)**</a>을 참조하세요. 

데이터를 훈련, 검증 및 검정 데이터 집합으로 분할했다면 이 과제를 완료한 것입니다.

In [ ]:
#add_your_task_3_2_code_here



### 과제 3.3: 훈련 작업 구성 및 실행

<a id="task3-3-continue"></a>

컨테이너, 추정기 및 하이퍼파라미터를 설정하여 첫 번째 훈련 작업을 시작합니다. 그런 다음 **fit()** 을 사용하여 모델을 훈련합니다. 더 자세한 보고서를 살펴보려면 **DebuggerHookConfig**를 사용하여 SageMaker Debugger를 사용하도록 설정합니다.

**힌트 1**: 버전이 **1.5-1**인 **XGBoost** 컨테이너를 사용합니다.

**힌트 2**: 훈련 작업을 시작하려면 **eta**, **gamma**, **max_depth**, **min_child_weight**, **num_round**, **objective** 및 **subsample**의 하이퍼파라미터를 설정합니다.

**힌트 3**: 앞에서 완료한 도전 과제 1에서 생성했던 **data_inputs**를 훈련 작업의 **inputs** 값으로 사용합니다.

**힌트 4**: inputs 및 experiment_config를 설정하여 훈련 작업을 구성할 수 있습니다. experiment_config에는 **sagemaker_session**, **run_name** 및 **experiment_name**이 포함되어야 합니다.

**힌트 5**: SageMaker Debugger를 사용하려는 경우에는 **DebuggerHookConfig**, **ProfilerConfig** 및 Debugger **rule** 객체를 구성합니다.

모델의 성능과 정확도에 가장 큰 영향을 줄 것으로 예상되는 하이퍼파라미터는 어느 것인가요? 어떤 하이퍼파라미터를 가장 먼저 튜닝할 계획인가요?

기본 훈련 작업을 구성하고 실행하는 방법에 관한 자세한 단계별 지침은 *부록* 섹션의 <a href="#task3-3-1" target="_self">**기본적인 훈련 작업 구성 및 실행(과제 3.3)**</a>을 참조하세요.

Debugger를 사용하도록 설정하여 훈련 작업을 구성 및 실행하며 보고서를 분석하는 방법에 관한 자세한 단계별 지침은 *부록* 섹션에서 <a href="#task3-3-2" target="_self">**SageMaker Debugger를 사용하도록 설정하여 훈련 작업을 구성 및 실행한 후 보고서 분석(과제 3.3)**</a>을 참조하세요.

훈련 작업을 하나 이상 완료했다면 이 과제를 완료한 것입니다.

In [ ]:
#add_your_task_3_3_code_here



### 과제 3.4: 하이퍼파라미터 튜닝

<a id="task3-4-continue"></a>

훈련 작업을 완료하고 분석했으므로 이번에는 분석 결과를 기준으로 하이퍼파라미터 범위를 튜닝하고 훈련 작업을 추가로 실행하여 모델을 개선합니다.

**힌트 1**: 튜닝을 시작하려면 **alpha**, **eta**, **max_depth**, **min_child_weight** 및 **num_round**의 하이퍼파라미터 범위를 설정합니다.

**힌트 2**: **HyperparameterTuner**를 실행할 때는 분석 결과에 따라 **objective_metric_name** 및 **objective_type**을 설정해야 합니다.

**힌트 3**: 튜닝 후 작업 결과가 SageMaker Autopilot 결과보다 개선되었는지 확인하려면 **Experiments**에서 **runs** 메뉴를 엽니다. 그리고 시험에서 **Metrics**를 확인합니다. **ObjectiveMetric**은 **F1** 점수인 **0.616**보다 높아야 하며 **validation:auc**의 **Final value**는 SageMaker Autopilot 점수인 **0.918**보다 커야 합니다.

하이퍼파라미터를 튜닝할 때 모델 성능을 가장 많이 개선한 항목은 어느 것이었나요?

훈련 하이퍼파라미터 범위를 구성하는 방법에 관한 자세한 단계별 지침은 *부록* 섹션의 <a href="#task3-4" target="_self">**훈련 하이퍼파라미터 범위 구성(과제 3.4)**</a>을 참조하세요.

훈련 하이퍼파라미터 범위를 구성하고 추가 훈련 작업을 시작했다면 이 과제를 완료한 것입니다.

In [ ]:
#add_your_task_3_4_code_here



## 도전 과제 4: 모델을 평가하여 편향 유무 확인

모델을 훈련했으므로 이제 Amazon SageMaker Clarify를 사용하여 모델을 평가합니다. 문제가 확인되면 탐지된 불균형 항목을 제거하고 모델을 재훈련할 수 있습니다.

이 과제를 완료하려면 다음 하위 과제를 완료해야 합니다.

- 훈련 작업에서 모델 생성
- SageMaker Clarify 모델 구성 생성
- SageMaker Clarify 편향 구성 생성
- SageMaker Clarify 프로세서 작업을 사용하여 편향, 데이터 및 모델 보고서 실행
- SageMaker Clarify를 사용하여 탐지된 불균형 항목 제거(선택 사항)
- 모델 재훈련(선택 사항)

이 문제는 완료하는 데 약 *80*분이 소요됩니다.

### 과제 4.1: 훈련 작업에서 모델 생성

<a id="task4-1-continue"></a>

**model_name**, **role** 및 **container_def**를 정의하여 XGBoost 모델을 생성하고 이름을 **create_model**로 지정합니다.

**힌트 1**: **xgb.create_model()** 을 호출하고 모델의 이름을 선택합니다.

**힌트 2**: session을 사용하여 **create_model**을 호출하고 **model_name**, **role** 및 **container_def**를 전달합니다.

모델 생성 방법에 관한 자세한 단계별 지침은 *부록* 섹션의 <a href="#task4-1" target="_self">**모델 생성(과제 4.1)**</a>을 참조하세요.

모델을 생성했다면 이 과제를 완료한 것입니다.

In [ ]:
#add_your_task_4_1_code_here



### 과제 4.2: SageMaker Clarify 모델 구성 생성

<a id="task4-2-continue"></a>

**SageMakerClarifyProcessor**를 사용하여 SageMaker Clarify 모델 구성을 생성합니다.

**힌트 1**: **instance_count** 및 **instance_type**을 설정합니다.

**힌트 2**: Capstone 실습을 시작할 때 생성했던 **role** 및 **session**을 사용합니다.

SageMaker Clarify 모델 구성을 생성하는 방법에 관한 자세한 단계별 지침은 *부록* 섹션의 <a href="#task4-2" target="_self">**SageMaker Clarify 모델 구성 생성(과제 4.2)**</a>을 참조하세요.

SageMaker Clarify 모델 구성을 생성했다면 이 과제를 완료한 것입니다.

In [ ]:
#add_your_task_4_2_code_here



### 과제 4.3: SageMaker Clarify 편향 구성 생성

<a id="task4-3-continue"></a>

데이터 구성, 모델 구성, 레이블 구성 및 편향 구성을 생성합니다.

**힌트 1**: 먼저 입력 경로, 출력 경로, 헤더 및 데이터 집합 유형을 설정하여 **DataConfig**부터 생성합니다.

**힌트 2**: 그런 다음, 콘텐츠 및 수락 유형, 모델 이름, 인스턴스 유형 및 인스턴스 수를 선택하여 **ModelConfig**를 생성합니다.

**힌트 3**: 다음으로는 확률 임계값을 설정하여 **ModelPredictedLabelConfig**를 생성합니다.

**힌트 4**: 마지막으로 레이블 값이나 임계값, 패싯 이름 및 패싯 값이나 임계값을 설정하여 **BiasConfig**를 생성합니다.

편향 보고서에서 어떤 패싯을 가장 먼저 살펴볼 예정인가요? 특히 편향가 발생하기 쉬운 특성이 있나요?

SageMaker Clarify 편향 구성을 생성하는 방법에 관한 자세한 단계별 지침은 *부록* 섹션의 <a href="#task4-3" target="_self">**SageMaker Clarify 편향 구성 생성(과제 4.3)**</a>을 참조하세요.

SageMaker Clarify 편향 구성을 생성했다면 이 과제를 완료한 것입니다.

In [ ]:
#add_your_task_4_3_code_here



### 과제 4.4: SageMaker Clarify 프로세서 작업을 사용하여 편향, 데이터 및 모델 보고서 실행

<a id="task4-4-continue"></a>

편향, 데이터 및 모델 보고서용 구성을 모두 선택했으므로 이제 보고서를 실행합니다.

**힌트 1**: **data_config**, **bias_config**, **model_predicted_label_config** 및 **model_config** 값을 **run_bias**로 전달합니다.

**힌트 2**: **pre_training_methods** 및 **post_training_methods**를 설정해야 합니다.

SageMaker Clarify를 사용하여 편향, 데이터 및 모델 보고서를 실행하는 방법에 관한 자세한 단계별 지침은 <a href="#task4-4" target="_self">**SageMaker Clarify를 사용하여 편향, 데이터 및 모델 보고서 실행(과제 4.4)**</a>을 참조하세요.

SageMaker Clarify 프로세서 작업을 사용하여 편향, 데이터 및 모델 보고서를 실행했다면 이 과제를 완료한 것입니다.

In [ ]:
#add_your_task_4_4_code_here



### 과제 4.5: SageMaker Clarify를 사용하여 탐지된 불균형 항목 제거(선택 사항)

<a id="task4-5-continue"></a>

SageMaker Clarify로 탐지한 불균형 항목은 여러 가지 방식으로 제거할 수 있습니다. 가장 편한 방법을 사용하면 됩니다. 이 실습에서는 열 중 하나에서 편향을 제거하는 Synthetic Minority Over-sampling Technique(SMOTE) 예제가 제공됩니다.

**힌트 1**: 불균형 항목을 제거한 후 테스트를 다시 실행하려면 다시 샘플링된 새 데이터 프레임을 생성합니다. 다음 과제에서 새 CSV 파일을 생성하여 업로드합니다.

불균형 항목을 제거하는 방법에 관한 자세한 단계별 지침은 *부록* 섹션의 <a href="#task4-5" target="_self">**불균형 항목 제거(과제 4.5)**</a>를 참조하세요.

SageMaker Clarify에서 탐지된 불균형 항목을 제거했다면 이 과제를 완료한 것입니다.

In [ ]:
#add_your_task_4_5_code_here



### 과제 4.6: 모델 재훈련(선택 사항)

<a id="task4-6-continue"></a>

Amazon S3에 새 파일을 업로드합니다. 그런 다음 새 추정기를 생성하고 새 데이터로 모델을 재훈련합니다.

**힌트 1**: **s3_client.upload_file**을 사용하여 버킷에 새 파일을 업로드합니다.

**힌트 2**: **xgboost_starter_script.py**를 사용하여 **XGBoost**를 호출합니다. 그런 다음 새 데이터로 모델을 재훈련합니다.

재훈련한 모델의 F1 점수가 더 높아졌나요? SageMaker Debugger를 사용했다면 검색된 모든 문제를 해결할 수 있었나요?

모델을 재훈련하는 방법에 관한 자세한 단계별 지침은 *부록* 섹션의 <a href="#task4-6" target="_self">**모델 재훈련(과제 4.6)**</a>을 참조하세요.

모델을 재훈련했다면 이 과제를 완료한 것입니다.

In [ ]:
#add_your_task_4_6_code_here



## 도전 과제 5: 배치 변환

이제 모델을 배포할 수 있습니다. 배치 레코드를 사용하여 배치 변환 작업을 수행한 후 Amazon S3에서 예측 및 정확도 데이터를 확인합니다. 그런 다음, SageMaker 인스턴스 중 몇 개를 정리합니다.

이 과제를 완료하려면 다음 하위 과제를 완료해야 합니다.

- 모델용 배치 변환 작업 생성
- Amazon S3에서 예측 데이터 확인
- SageMaker 인스턴스 정리(선택 사항)

이 문제는 완료하는 데 약 *40*분이 소요됩니다.

### 과제 5.1: 모델용 배치 변환 작업 생성

<a id="task5-1-continue"></a>

모델 추정기에서 **transformer**를 사용하여 배치 변환 작업을 생성합니다. 그런 다음, 배치 작업을 실행합니다.

**힌트 1**: 변환기를 사용하여 **instance_count**, **instance_type**, **strategy**, **assemble_with** 및 **output_path**를 구성합니다.

**힌트 2**: **test_path**에 나와 있는 검정 데이터를 엔드포인트로 전송한 후 결과가 표시될 때까지 기다립니다.

배치 변환 작업을 생성하는 방법에 관한 자세한 단계별 지침은 *부록* 섹션의 <a href="#task5-1" target="_self">**배치 변환 작업 생성(과제 5.1)**</a>을 참조하세요.

배치 변환 작업을 생성하고 레코드 집합을 사용하여 해당 작업을 실행했다면 이 과제를 완료한 것입니다.

In [ ]:
#add_your_task_5_1_code_here



### 과제 5.2: Amazon S3에서 예측 데이터 확인

<a id="task5-2-continue"></a>

배치 변환 작업이 완료되면 Amazon S3에서 데이터를 읽습니다. 

**힌트 1**: **%aws s3 cp --recursive $transformer.output_path ./** 를 사용하면 변환기 출력에서 데이터를 복사할 수 있습니다.

**힌트 2**: 복사한 데이터는 **%head test_data_batch.csv.out**을 사용하여 확인할 수 있습니다.

예측을 살펴봅니다. 예상치 못한 항목이 있나요?

배치 변환 작업에서 예측 데이터를 확인하는 방법에 관한 자세한 단계별 지침은 *부록* 섹션의 <a href="#task5-2" target="_self">**배치 변환 작업에서 예측 데이터 확인(과제 5.2)**</a>을 참조하세요.

배치 변환 작업에서 예측 데이터를 확인했다면 이 과제를 완료한 것입니다.

In [ ]:
#add_your_task_5_2_code_here



### 과제 5.3: SageMaker 인스턴스 정리(선택 사항)

비용을 줄이려면 더 이상 사용하지 않는 인스턴스를 삭제하는 것이 좋습니다. SageMaker Studio를 사용하면 인스턴스를 빠르게 삭제할 수 있습니다. SageMaker Studio에서 현재 리소스 목록을 연 다음, 남아 있는 인스턴스를 모두 닫습니다.

다음 파이프라인 과제를 완료할 계획이라면 **노트북 인스턴스를 실행 상태로 두세요**.

**힌트 1**: SageMaker Studio에서 **Running Terminals and Kernels** 아이콘을 선택하면 실행 인스턴스 목록을 확인할 수 있습니다.

**힌트 2**: **Shut down** 아이콘을 사용하여 인스턴스를 중지할 수 있습니다.

<a id="task5-3-continue"></a>

SageMaker Studio에서 SageMaker 인스턴스를 정리하는 방법에 관한 자세한 단계별 지침은 *부록* 섹션의 <a href="#task5-3" target="_self">**SageMaker Studio에서 SageMaker 인스턴스 정리(과제 5.3)**</a>를 참조하세요.

SageMaker Studio에서 SageMaker 인스턴스를 모두 중지했다면 이 과제를 완료한 것입니다.

## 도전 과제 6: 자동 파이프라인 빌드(선택 사항)

기계 학습 워크플로에 SageMaker SDK와 SageMaker Studio를 사용해 보았으므로 이제 SageMaker 파이프라인을 사용하여 워크플로 크기를 조정합니다. 이 도전 과제를 완료하려면 실습 환경에서 제공되는 파이프라인 스크립트를 단계별로 진행합니다. 

- 파이프라인 단계 생성
    - SageMaker Feature Store에서 처리된 데이터 쿼리
    - 모델 훈련 및 튜닝
    - 훈련한 모델 평가
    - 배치 변환 작업 수행
    - 모델 등록
    - SageMaker Clarify를 사용하여 모델 훈련 평가
- 파이프라인 정의 및 시작
- ML 계보 추적 확인

이 문제는 완료하는 데 약 *120*분이 소요됩니다.


### 과제 6.1: 파이프라인 구성

<a id="task6-1-continue"></a>

파이프라인 템플릿을 사용하여 입력과 출력을 구성합니다. 구성을 완료한 후 파이프라인을 실행합니다. 파이프라인에는 광범위한 단계가 포함될 수 있습니다. 구성할 수 있는 단계의 제안 목록이 아래에 나와 있습니다.
- **AutoModelProcess**: .csv 파일을 끌어온 후 훈련, 검정, 검증 데이터 집합으로 분할하는 **Processing** 단계입니다.
- **AutoHyperParameterTuning**: 여러 하이퍼파라미터를 사용하여 모델을 튜닝하는 **Tuning** 단계입니다.
- **AutoEvalBestModel**: 최적 모델을 설명하기 위한 평가 보고서를 생성하는 **Processing** 단계입니다.
- **CheckAUCScoreAutoEvaluation**: 평가 지표에 따라 모델을 평가하는 **Condition** 단계입니다. 
- **AutoCreateModel**: 모델을 생성하는 **Model** 단계입니다.
- **RegisterAutoModel-RegisterModel**: 모델을 등록하는 **RegisterModel** 단계입니다.
- **AutoModelConfigFile**: 편향 보고서를 생성하는 **Processing** 단계입니다.
- **AutoTransform**: 배치 변환 작업을 실행하는 **Transform** 단계입니다.
- **ClarifyProcessingStep**: SageMaker Clarify 작업을 실행하는 **Processing** 단계입니다.

**힌트 1**: 여러 파이프라인 단계 중에서 원하는 단계를 선택할 수 있습니다. 파이프라인 단계에 대해 자세히 알아보고 각 단계의 샘플 코드를 확인하려면 *추가 리소스* 섹션에서 *파이프라인 단계 빌드 및 관리* 문서를 참조하세요.

**힌트 2**: 먼저 **Processing** 단계부터 시작하여 데이터를 끌어옵니다. 그런 다음 **Tuning** 단계를 생성하여 모델을 튜닝합니다. 다음으로는 **Model** 단계를 생성하여 모델을 생성합니다.

**힌트 3**: 세부 단계에 포함된 샘플 솔루션에서는 평가 보고서와 편향 보고서 생성, 배치 변환 작업과 SageMaker Clarify 작업 실행 단계가 포함됩니다. 

파이프라인을 구성하는 방법에 관한 자세한 단계별 지침은 *부록* 섹션의 <a href="#task6-1" target="_self">**파이프라인 구성(과제 6.1)**</a>을 참조하세요.

파이프라인을 설정하고 파이프라인 작업을 시작했다면 이 과제를 완료한 것입니다.

In [ ]:
#add_your_task_6_1_code_here



### 과제 6.2: 파이프라인 모니터링

<a id="task6-2-continue"></a>

실행 중인 파이프라인을 모니터링하면서 입력과 출력을 확인합니다. 

**힌트 1**: `RunPipeline.describe()`를 사용하여 방금 생성한 파이프라인을 설명합니다.

**힌트 2**: SageMaker Studio UI에서 실행 중인 파이프라인 단계를 확인할 수 있습니다. **SageMaker resources** 메뉴를 열고 **Pipelines**를 선택한 후 방금 생성한 파이프라인을 선택합니다. 

파이프라인을 모니터링하는 방법에 관한 자세한 단계별 지침은 *부록* 섹션의 <a href="#task6-2" target="_self">**파이프라인 모니터링**</a>을 참조하세요.

파이프라인 모니터링을 완료했다면 이 과제를 완료한 것입니다.

In [ ]:
#add_your_task_6_2_code_here



축하합니다! 자동차 보험 데이터 집합을 사용하여 사기일 가능성이 있는 보험금 청구 건을 탐지했습니다. 그리고 SageMaker Studio 및 Amazon SageMaker Python SDK를 사용하여 특정 자동차 보험금 청구 건이 사기일 가능성을 예측하는 기술 솔루션을 살펴보았습니다.

### 정리

이 노트북을 완료했습니다. 실습의 다음 부분으로 이동하려면 다음을 수행합니다.

- 이 노트북 파일을 닫습니다.
- 실습 지침으로 돌아갑니다.

## 추가 리소스

- [Autopilot 지표](https://docs.aws.amazon.com/sagemaker/latest/dg/autopilot-metrics-validation.html)
- [처리 단계](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-processing)

## 부록

<a name="task1-1" id="task1-1"></a>

### 부록: 데이터 검토(과제 1.1)

데이터를 검토하려면 경로를 지정하고 Pandas를 사용하여 데이터를 로드합니다. 두 테이블의 샘플을 잠시 검토합니다.

In [ ]:
#read-csv-files
claims_data = pd.read_csv("./data/claims_preprocessed.csv", index_col=0)
customers_data = pd.read_csv("./data/customers_preprocessed.csv", index_col=0)

In [ ]:
#claims-data-sample
claims_data.head()

In [ ]:
#customers-data-sample
customers_data.head()

이 실습을 계속 진행하려면 <a href="#task1-1-continue" target="_self">과제 1.1</a>로 돌아갑니다.

<a name="task1-2-1" id="task1-2-1"></a>

### 부록: SageMaker Studio에서 데이터 집합 탐색(과제 1.2)

SageMaker Data Wrangler에서 데이터 탐색을 시작합니다. S3 버킷에서 파일을 가져와 데이터를 분석합니다.

다음 단계를 수행하면 SageMake Studio에서 새 탭이 열립니다. 여기서 설명하는 지침에 따라 작업을 진행하려면 다음 옵션 중 하나를 사용하세요.
- **옵션 1**: 탭을 나란히 표시합니다. 주 SageMaker Studio 창에서 분할 화면 보기를 생성하려면 **capstone_ko_kr.ipynb** 탭을 옆쪽으로 끌거나 **capstone_ko_kr.ipynb** 탭을 마우스 오른쪽 버튼으로 클릭하여 선택한 후 **New View for Notebook**을 선택합니다. 그러면 특성 그룹을 살펴볼 때 지침을 확인할 수 있습니다.
- **옵션 2**: SageMaker Studio 탭을 서로 전환하면서 지침에 따라 작업을 진행합니다.

1. SageMaker Studio 왼쪽에서 **Home** 아이콘을 선택합니다.
1. **Data** 섹션을 확장한 다음 **Data Wrangler**를 선택합니다.

SageMaker Studio에서 **Data Wrangler** 탭이 열립니다.

1. **+** **Create Data Wrangler flow**을 선택합니다.

SageMaker Studio에서 **untitled.flow** 탭이 열립니다.

1. **untitled.flow** 탭이 로드되어 진행률 표시줄이 표시될 때까지 기다립니다. 2–3분 정도 걸릴 수 있습니다.

SageMaker Studio에서 *Data Wrangler* 탭 내에 **연결 생성** 페이지가 열립니다.

1. **untitled.flow** 파일 탭에서 마우스 오른쪽 버튼을 클릭해 컨텍스트 메뉴를 연 다음, 파일 이름을 변경하려면 **Rename Data Wrangler Flow...** 를 선택합니다.

SageMaker Studio에서 **파일 이름 바꾸기** 메시지 창이 열립니다.

1. **새 이름**에 `CapstoneDataWrangler.flow`를 입력합니다.
1. <span style="background-color:#57c4f8; font-size:90%;  color:black; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#00a0d2; border-radius:2px; margin-right:5px; white-space:nowrap">Rename</span>을 선택합니다.

**파일 이름 바꾸기** 메시지 창이 닫힙니다.

1. *CapstoneDataWrangler.flow* 탭의 **Data sources** 섹션에서 **Amazon S3**를 선택합니다.

SageMaker Studio는 *DataWrangler.flow* 탭 내부의 **S3에서 데이터 집합 가져오기** 페이지를 엽니다.

1. 버킷 목록에서 이름에 **databucket**이 포함된 버킷을 엽니다.
1. **claims.csv**라는 이름의 파일인 첫 번째 데이터 집합을 선택합니다.

1. <span style="background-color:#57c4f8; font-size:90%;  color:black; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#00a0d2; border-radius:2px; margin-right:5px; white-space:nowrap">Import</span>를 선택합니다.

1. *CapstoneDataWrangler.flow* 탭의 왼쪽 상단에 있는 **&lt; Data flow**를 선택해서 **Data flow** 보기로 돌아갑니다.
1. *CapstoneDataWrangler.flow* 탭의 왼쪽 상단에 있는 **Import** 탭을 선택합니다.

SageMaker Studio에서 **연결 생성** 페이지가 열립니다.

1. *CapstoneDataWrangler.flow* 탭의 **Data sources** 섹션에서 **Amazon S3**를 선택합니다.

SageMaker Studio는 *DataWrangler.flow* 탭 내부의 **S3에서 데이터 집합 가져오기** 페이지를 엽니다.

1. 버킷 목록에서 이름에 **databucket**이 포함된 버킷을 엽니다.
1. **customers.csv**라는 파일인 두 번째 데이터 집합을 선택합니다.
1. <span style="background-color:#57c4f8; font-size:90%;  color:black; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#00a0d2; border-radius:2px; margin-right:5px; white-space:nowrap">Import</span>를 선택합니다.

1. *CapstoneDataWrangler.flow* 탭의 왼쪽 상단에 있는 **&lt; Data flow**를 선택해서 **Data flow** 보기로 돌아갑니다.

1. Data flow 탭에서 **Data types** 아이콘 옆의 **+** 기호를 선택하고 **Get data insights**를 선택합니다.

1. 보고서를 생성하고 인사이트를 살펴봅니다.

1. **Data flow** 탭으로 돌아와 **Data types** 아이콘 옆의 **+** 기호를 선택하고 **Add analysis**를 선택합니다.

1. 분석을 생성하고 결과를 살펴봅니다.

데이터 집합을 자세히 살펴볼 수 있는 보고서 유형이라면 무엇이든 사용할 수 있습니다. 결과 확인을 완료한 후 다음 과제를 계속 진행할 수 있습니다.

이 실습을 계속 진행하려면 <a href="#task1-2-continue" target="_self">과제 1.2</a>로 돌아갑니다.


<a name="task1-2-2" id="task1-2-2"></a>

### 부록: 노트북에서 데이터 집합 탐색(과제 1.2)

여러 가지 방식으로 데이터 집합을 탐색할 수 있습니다. 수행 가능한 몇 가지 데이터 탐색 단계의 다양한 예제가 아래에 나와 있습니다. 이러한 예제를 참조하여 데이터 집합의 여러 측면부터 먼저 탐색하면 됩니다.

In [ ]:
#gender-graph
import matplotlib.pyplot as plt
customers_data.customer_gender_female.value_counts(normalize=True).plot.bar()
plt.xticks([0, 1], ["Male", "Female"]);

In [ ]:
#fraud-graph
claims_data.fraud.value_counts(normalize=True).plot.bar()
plt.xticks([0, 1], ["Not Fraud", "Fraud"]);

In [ ]:
#education-category-graphs
educ = customers_data.customer_education.value_counts(normalize=True, sort=False)
plt.bar(educ.index, educ.values)
plt.xlabel("Customer Education Level");

In [ ]:
#claim-amount-graph
plt.hist(claims_data.total_claim_amount, bins=30)
plt.xlabel("Total Claim Amount")

In [ ]:
#claims-filed-graph
customers_data.num_claims_past_year.hist(density=True)
plt.suptitle("Number of Claims in the Past Year")
plt.xlabel("Number of claims per year")

In [ ]:
#paid-plot-graphs
sns.pairplot(
    data=customers_data, vars=["num_insurers_past_5_years", "months_as_customer", "customer_age"]
);

In [ ]:
#fraud-insurers-graph
combined_data = customers_data.join(claims_data)
sns.lineplot(x="num_insurers_past_5_years", y="fraud", data=combined_data);

In [ ]:
#months-as-customer-graph
sns.boxplot(x=customers_data["months_as_customer"]);

In [ ]:
#customer-age-graph
sns.boxplot(x=customers_data["customer_age"]);

In [ ]:
#fraud-gender-graph
combined_data.groupby("customer_gender_female").mean()["fraud"].plot.bar()
plt.xticks([0, 1], ["Male", "Female"])
plt.suptitle("Fraud by Gender");

In [ ]:
#correlation-matrix-graph
cols = [
    "fraud",
    "customer_gender_male",
    "customer_gender_female",
    "months_as_customer",
    "num_insurers_past_5_years",
]
corr = combined_data[cols].corr()

# plot the correlation matrix
sns.heatmap(corr, annot=True, cmap="Reds");

In [ ]:
#load-combined-data
combined_data = pd.read_csv("./data/claims_customer.csv")

In [ ]:
#remove-unnecessary-columns
combined_data = combined_data.loc[:, ~combined_data.columns.str.contains("^Unnamed: 0")]
combined_data.head()

In [ ]:
#describe-combined-data
combined_data.describe()

In [ ]:
#generate-statistics
combined_stats = []

for col in combined_data.columns:
    combined_stats.append(
        (
            col,
            combined_data[col].nunique(),
            combined_data[col].isnull().sum() * 100 / combined_data.shape[0],
            combined_data[col].value_counts(normalize=True, dropna=False).values[0] * 100,
            combined_data[col].dtype,
        )
    )

stats_df = pd.DataFrame(
    combined_stats,
    columns=["feature", "unique_values", "percent_missing", "percent_largest_category", "datatype"],
)
stats_df.sort_values("percent_largest_category", ascending=False)

In [ ]:
#heatmap-graph
sns.set_style("white")

corr_list = [
    "customer_age",
    "months_as_customer",
    "total_claim_amount",
    "injury_claim",
    "vehicle_claim",
    "incident_severity",
    "fraud",
]

corr_df = combined_data[corr_list]
corr = round(corr_df.corr(), 2)

fix, ax = plt.subplots(figsize=(15, 15))

mask = np.zeros_like(corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True

ax = sns.heatmap(corr, mask=mask, ax=ax, annot=True, cmap="OrRd")

ax.set_xticklabels(ax.xaxis.get_ticklabels(), fontsize=10, ha="right", rotation=45)
ax.set_yticklabels(ax.yaxis.get_ticklabels(), fontsize=10, va="center", rotation=0)

plt.show()

이 실습을 계속 진행하려면 <a href="#task1-2-continue" target="_self">과제 1.2</a>로 돌아갑니다.

<a name="task1-3-1" id="task1-3-1"></a>

### 부록: SageMaker Studio에서 테이블 조인(과제 1.3)

1. *CapstoneDataWrangler.flow* 탭의 왼쪽 상단에 있는 **&lt; Data flow**를 선택해서 **Data flow** 보기로 돌아갑니다.
1. **claims.CSV Data types** 아이콘 옆에 있는 **+** 기호를 선택하고 컨텍스트 메뉴에서 **Join**을 선택합니다.

SageMaker Data Wrangler에서 **Join** 페이지가 표시됩니다.

1. **customers.csv Data types** 아이콘을 선택합니다.
1. <span style="background-color:#1a1b22; font-size:90%; color:#57c4f8; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#57c4f8; border-width:thin; border-style:solid; border-radius:2px; margin-right:5px; white-space:nowrap">Configure</span>를 선택합니다.
1. **Join Type**에서 **Inner**를 선택합니다.
1. **Columns** 섹션에서

    - **Left**의 경우, <span style="background-color:#1a1b22; font-size:90%; color:#57c4f8; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#57c4f8; border-width:thin; border-style:solid; border-radius:2px; margin-right:5px; white-space:nowrap">policy_id</span>를 선택합니다.
    
    - **Right**의 경우, <span style="background-color:#1a1b22; font-size:90%; color:#57c4f8; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#57c4f8; border-width:thin; border-style:solid; border-radius:2px; margin-right:5px; white-space:nowrap">policy_id</span>를 선택합니다.

1. <span style="background-color:#1a1b22; font-size:90%; color:#57c4f8; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#57c4f8; border-width:thin; border-style:solid; border-radius:2px; margin-right:5px; white-space:nowrap">Preview</span>를 선택합니다.

1. <span style="background-color:#57c4f8; font-size:90%;  color:black; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#00a0d2; border-radius:2px; margin-right:5px; white-space:nowrap">Add</span>를 선택합니다.

이 실습을 계속 진행하려면 <a href="#task1-3-continue" target="_self">과제 1.3</a>으로 돌아갑니다.

<a name="task1-3-2" id="task1-3-2"></a>

### 부록: 훈련 전 편향 보고서 실행(과제 1.3)

SageMaker Data Wrangler 흐름을 사용하여 SageMaker Clarify 편향 보고서를 생성합니다.

1. **CapstoneDataWrangler.flow** 탭을 선택합니다.
1. 필요하다면 *DataWranglerLab.flow* 탭의 왼쪽 위에 있는 **&lt; Data flow**를 택해서 **Data flow** 보기로 돌아갑니다. 
1. **Join** 아이콘 옆에 있는 **+** 기호를 선택하고 컨텍스트 메뉴에서 **Add analysis**를 선택합니다.
1. **Create analysis** 섹션에서

- **Analysis type**에는 **Bias Report**를 선택합니다.
- **Analysis name**에는 `fraud bias by age`를 입력합니다.
- **Select the column your model predicts (target)** 에는 **fraud**를 선택합니다.
- **Is your predicted column a value or threshold?** 에는 **value** 옵션을 선택합니다.
- **Predicted value(s)** 에는 **1**을 입력합니다.
- **Select the column to analyze for bias**에는 **customer_age**를 선택합니다.

1. <span style="background-color:#1a1b22; font-size:90%; color:#57c4f8; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#57c4f8; border-width:thin; border-style:solid; border-radius:2px; margin-right:5px; white-space:nowrap">Run</span>를 선택합니다.

작업이 완료되면 반환된 지표를 확인합니다. 편향 유무를 확인한 후 분석한 열에서 수행할 처리 단계를 계획합니다. 

편향 유무를 분석할 모든 열에서 이러한 단계를 반복할 수 있습니다.

이 실습을 계속 진행하려면 <a href="#task1-3-continue" target="_self">과제 1.3</a>으로 돌아갑니다.

<a name="task1-4-1" id="task1-4-1"></a>

### 부록: SageMaker Data Wrangler를 사용하여 데이터 준비(과제 1.4)

데이터 집합을 결합하고 SageMaker Data Wrangler에서 **policy_id**를 사용하여 데이터 집합을 조인합니다.

SageMaker Data Wrangler를 사용하면 흐름의 어떤 지점에서나 데이터를 조인할 수 있습니다. 개별 파일을 조인하기 전에 데이터 준비를 완료할 수도 있고 조인 후에 특성을 변환할 수도 있습니다. 즉, SageMaker Data Wrangler 흐름은 유동적으로 활용할 수 있습니다.

과제 1.3에서 테이블을 조인하지 않았다면 아래 지침에 따라 조인을 수행합니다.

1. *CapstoneDataWrangler.flow* 탭의 왼쪽 상단에 있는 **&lt; Data flow**를 선택해서 **Data flow** 보기로 돌아갑니다.
1. 첫 번째 **Data types** 아이콘 옆에 있는 **+** 기호를 선택하고 컨텍스트 메뉴에서 **Join**을 선택합니다.

SageMaker Data Wrangler에서 **Join** 페이지가 표시됩니다.

1. 두 번째 **Data types** 아이콘을 선택합니다.
1. <span style="background-color:#1a1b22; font-size:90%; color:#57c4f8; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#57c4f8; border-width:thin; border-style:solid; border-radius:2px; margin-right:5px; white-space:nowrap">Configure</span>를 선택합니다.
1. **Join Type**에서 **Inner**를 선택합니다.
1. **Columns** 섹션에서

    - **Left**의 경우, <span style="background-color:#1a1b22; font-size:90%; color:#57c4f8; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#57c4f8; border-width:thin; border-style:solid; border-radius:2px; margin-right:5px; white-space:nowrap">policy_id</span>를 선택합니다.
    
    - **Right**의 경우, <span style="background-color:#1a1b22; font-size:90%; color:#57c4f8; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#57c4f8; border-width:thin; border-style:solid; border-radius:2px; margin-right:5px; white-space:nowrap">policy_id</span>를 선택합니다.

1. <span style="background-color:#1a1b22; font-size:90%; color:#57c4f8; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#57c4f8; border-width:thin; border-style:solid; border-radius:2px; margin-right:5px; white-space:nowrap">Preview</span>를 선택합니다.

1. <span style="background-color:#57c4f8; font-size:90%;  color:black; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#00a0d2; border-radius:2px; margin-right:5px; white-space:nowrap">Add</span>를 선택합니다.

데이터 테이블이 조인된 상태에서 결합된 데이터를 변환합니다.

1. *CapstoneDataWrangler.flow* 탭의 왼쪽 상단에 있는 **&lt; Data flow**를 선택해서 **Data flow** 보기로 돌아갑니다.
1. **Join** 아이콘 옆에 있는 **+** 기호를 선택하고 컨텍스트 메뉴에서 **Add transform**을 선택합니다.

이 메뉴를 사용하여 데이터 세트에 여러 변환을 추가할 수 있습니다. 변환 메뉴 왼쪽에 데이터 집합 미리 보기가 표시됩니다.

SageMaker Data Wrangler 흐름에 다음 변환 단계를 추가합니다.
- 범주형 인코딩(원-핫 인코딩): **authorities_contacted**, **collision_type**, **customer_gender**, **driver_relationship**, **incident_type**, **policy_state**(**Skip** 무효 처리 전략 사용, 출력 스타일은 **Columns** 선택).
- 범주형 인코딩(서수형 인코딩): **customer_education**, **incident_severity**, **police_report_available**, **policy_liability**(**Skip** 무효 처리 전략 사용)
- 열 구문 분석 유형: **vehicle_claim** 및 **total_claim_amount**를 **Float**에서 **Long**으로 구문 분석
- 열 관리(열 삭제): **customer_zip** and **policy_id_1**.
- 열 관리(열 이동): **fraud**(**Move to start** 사용).
- 열 관리(열 이름 바꾸기): **collision_type_N/A** 및 **driver_relationship_N/A**에서 **/** 기호를 **_** 로 바꾸기
- 열 관리(열 이름 바꾸기): **policy_id_0**의 이름을 **policy_id**로 바꾸기

이름에 **/** 문자가 포함된 열이 있다면 **/** 를 **_** 로 바꿔 열 이름을 변경합니다. 열 이름에 공백 문자가 있는 경우 공백을 **_** 로 바꿔 열 이름을 변경합니다. 예를 들어 원-핫 인코딩을 사용하여 생성했으며 값이 **N/A**인 모든 열은 이름을 바꿔야 합니다. SageMaker Feature Store에서는 이름에 **/** 문자 또는 공백이 포함된 열을 허용하지 않습니다.

데이터를 변환했으며 모델 훈련을 시작할 준비가 되었다면 다음 과제를 계속 진행할 수 있습니다. 언제든지 이 흐름으로 돌아와 훈련과 튜닝 과정에서 확인된 사항을 토대로 변경을 수행할 수 있습니다.


이 실습을 계속 진행하려면 <a href="#task1-4-continue" target="_self">과제 1.4</a>로 돌아갑니다.

<a name="task1-4-2" id="task1-4-2"></a>

### 부록: 처리된 데이터의 예제 집합 가져오기(과제 1.4)

전처리 중에 문제가 발생하거나 이미 처리되어 있는 데이터 집합을 로드하려는 경우, S3 버킷의 데이터 폴더에 저장되어 있는 처리된 데이터에 액세스합니다.

In [ ]:
#processed-data-import
s3_client.upload_file(Filename="data/claims_customer.csv", Bucket=bucket, Key=f"{prefix}/data/raw/claims_customer.csv")
df_processed = pd.read_csv("./data/claims_customer.csv", index_col=None)
df_processed.head()

이 실습을 계속 진행하려면 <a href="#task1-4-continue" target="_self">과제 1.4</a>로 돌아갑니다.

<a name="task2-1" id="task2-1"></a>

### 부록: Export to 옵션을 사용하여 특성 그룹 생성(과제 2.1)

SageMaker Data Wrangler에서는 SageMaker Feature Store로 데이터를 내보낼 수 있습니다. Data Wrangler는 특성 그룹을 구성하는 데 필요한 모든 코드가 들어 있는 노트북을 생성하며, 변환된 데이터를 특성 그룹에 수집합니다.

1. **Add step**을 선택합니다.

1. **Custom transform**을 선택합니다.

1. **Name**에 `event_time`을 입력합니다. 

1. **Python (PySpark)** 가 아직 선택되어 있지 않다면 선택합니다.

1. **Your custom transform**에 아래 코드를 입력합니다.


In [ ]:
import time
import datetime
from pyspark.sql.functions import lit
date_time = datetime.date.today()

df = df.withColumn("event_time", lit(time.mktime(date_time.timetuple())))


1. <span style="background-color:#1a1b22; font-size:90%; color:#57c4f8; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#57c4f8; border-width:thin; border-style:solid; border-radius:2px; margin-right:5px; white-space:nowrap">Preview</span>를 선택합니다.

1. <span style="background-color:#57c4f8; font-size:90%;  color:black; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#00a0d2; border-radius:2px; margin-right:5px; white-space:nowrap">Add</span>를 선택합니다.

그러면 **event_time**이 데이터 집합에 열로 추가됩니다. SageMaker Feature Store를 사용하려면 **event_time**과 고유한 **record** ID가 필요합니다. **record** ID로는 **policy_id**를 사용합니다.

1. 데이터 흐름으로 돌아가려면 **&lt; Data flow** 아이콘을 선택합니다.

1. SageMaker Data Wrangler에서 변환 옆의 **+** 아이콘을 선택합니다.

1. **Export to**를 선택합니다.

1. **SageMaker Feature Store (via Jupyter Notebook)** 를 선택합니다.

새 노트북이 열립니다.

1. 첫 번째 셀에서 다음 변수를 변경합니다.
- **record_identifier_feature_name**에서 **None**을 `'policy_id'`로 바꿉니다. customers 테이블과 claims 테이블을 조인했는데 두 번째 **policy_id** 열을 삭제하지 않았다면 **None**을 `'policy_id_0'`으로 바꿔야 할 수 있습니다. **if** 문 다음에 위치하는 **None** 값은 변경하지 마세요.
- **event_time_feature_name**에서 **None**을 `'event_time'`으로 바꿉니다. **if** 문 다음에 위치하는 **None** 값은 변경하지 마세요.

**예상 출력:** 셀 편집을 완료하면 다음 예와 유사해야 합니다.


In [ ]:
record_identifier_feature_name = "policy_id"
if record_identifier_feature_name is None:
   raise SystemExit("Select a column name as the feature group record identifier.")

event_time_feature_name = "event_time"
if event_time_feature_name is None:
   raise SystemExit("Select a column name as the event time feature name.")


2. 모든 셀을 실행하여 특성 정의와 특성 그룹을 생성하고, 처리 작업을 사용하여 변환된 데이터를 특성 그룹에 수집합니다. 

셀 실행이 완료되면 특성 저장소를 사용할 수 있습니다.

이 실습을 계속 진행하려면 <a href="#task2-1-continue" target="_self">과제 2.1</a>로 돌아갑니다.

<a name="task2-2" id="task2-2"></a>

### 부록: Athena를 사용하여 오프라인 저장소에서 레코드 추출(과제 2.2)

**athena_query**를 사용하여 Athena 쿼리를 설정합니다. 그런 다음 **query_string**을 설정합니다. 마지막으로 쿼리를 실행하고 결과의 샘플을 확인합니다.

In [ ]:
#configure-and-run-athena-query
try:
    # If there is a feature group, get the name
    feature_group_name = sagemaker_session.boto_session.client("sagemaker", region_name=region).list_feature_groups()['FeatureGroupSummaries'][0]['FeatureGroupName']
    feature_group = FeatureGroup(name=feature_group_name, sagemaker_session=sagemaker_session)

    # Confirm the Athena settings are configured
    try:
        boto3.client('athena').update_work_group(
            WorkGroup='primary',
            ConfigurationUpdates={
                'EnforceWorkGroupConfiguration':False
            }
        )
    except Exception:
        pass
    
    # Configure the query
    query = feature_group.athena_query()
    table = query.table_name
    query_string = f'SELECT * FROM "{table}" '
    output_location = f"s3://{sagemaker_session.default_bucket()}/query_results/"
    print(f"Athena query output location: \n{output_location}")

    # Run the query
    query.run(query_string=query_string, output_location=output_location)
    query.wait()
    df_feature_store = query.as_dataframe()
    
    # Wait for data to appear in the feature group
    attempts = 0
    while len(df_feature_store.index) == 0 and attempts < 30:
        print("Waiting for feature group to populate...")
        time.sleep(60)
        # Rerun the query
        query.run(query_string=query_string, output_location=output_location)
        query.wait()
        df_feature_store = query.as_dataframe()
        # Increment the attempts
        attempts += 1
    if len(df_feature_store.index) != 0:
        print("The feature group is populated.")
except IndexError as e:
    # If there is no feature group, thrown an error
    print("No feature groups were found. Please create a feature group.")

이전 코드 블록을 실행한 후 Amazon Athena 쿼리 레코드는 이름이 *sagemaker*로 시작하는 Amazon S3 버킷에 저장됩니다. 저장된 쿼리 객체는 **query_results**라는 디렉터리에 있습니다.

이 실습을 계속 진행하려면 <a href="#task2-2-continue" target="_self">과제 2.2</a>로 돌아갑니다.

<a name="task3-1" id="task3-1"></a>

### 부록: 실험 및 시험 이름 지정(과제 3.1)

실험을 생성하려면 **sagemaker.experiments.run** 라이브러리를 사용하세요. **experiment_name**, **run_name** 및 **description**을 설정합니다.

In [ ]:
from time import gmtime, strftime

#create experiment and run-names
create_date = strftime("%m%d%H%M")
capstone_experiment_name="capstone-experiment-{}".format(create_date)
capstone_run_name = "lab-capstone-run-{}".format(create_date)

# define a run_tag
run_tags = [{'Key': 'lab-capstone', 'Value': 'lab-capstone-run'}]

# provide a description
description="Using SM Experiments with the Auto dataset."

print(f"Experiment name - {capstone_experiment_name},  run name - {capstone_run_name}")

이 실습을 계속 진행하려면 <a href="#task3-1-continue" target="_self">과제 3.1</a>로 돌아갑니다.

<a name="task3-2" id="task3-2"></a>

### 부록: 데이터를 훈련, 검정 및 검증 데이터 집합으로 분할(과제 3.2)

데이터를 분할하려면 **np.split**을 사용하여 원하는 데이터 분할 방식을 지정합니다. 그 후에는 CSV 파일을 생성하여 S3 버킷에 업로드합니다. 다음으로는 훈련 입력을 설정합니다. 마지막으로 **data_inputs** 변수를 생성합니다. data_inputs는 모델을 훈련할 때 훈련 및 검증 데이터 집합을 지정하기 위해 도전 과제 전반에 걸쳐 사용합니다.

In [ ]:
#train-validation-test-split
try:
    # If there is a feature group, use it
    df_feature_store = df_feature_store.iloc[: , :-4]
    df_processed_pre_split = df_feature_store
    print("Using the records from the feature group")
except NameError:
    # If there is no feature group, use the processed dataset
    df_processed = pd.read_csv("./data/claims_customer.csv", index_col=None)
    df_processed_pre_split = df_processed
    print("Using the processed records from Amazon S3")

# Split the data into train, validation, and test datasets
train_data, validation_data, test_data = np.split(
    df_processed_pre_split.sample(frac=1, random_state=1729),
    [int(0.7 * len(df_processed_pre_split)), int(0.9 * len(df_processed_pre_split))],
)

# Create the CSV files and upload them to your default bucket
train_data.to_csv("train_data.csv", index=False, header=False)
validation_data.to_csv("validation_data.csv", index=False, header=False)
test_data.to_csv("test_data.csv", index=False, header=False)

train_path = S3Uploader.upload("train_data.csv", "s3://{}/{}".format(bucket, prefix))
validation_path = S3Uploader.upload("validation_data.csv", "s3://{}/{}".format(bucket, prefix))
test_path = S3Uploader.upload("test_data.csv", "s3://{}/{}".format(bucket, prefix))

# Set the training inputs
train_input = TrainingInput(train_path, content_type="text/csv")
validation_input = TrainingInput(validation_path, content_type="text/csv")
test_input = TrainingInput(test_path, content_type="text/csv")

data_inputs = {
    "train": train_input,
    "validation": validation_input
}

이 실습을 계속 진행하려면 <a href="#task3-2-continue" target="_self">과제 3.2</a>로 돌아갑니다.

<a name="task3-3-1" id="task3-3-1"></a>

### 부록: 기본적인 훈련 작업 구성 및 실행(과제 3.3)

기본적인 훈련 작업을 시작하려는 경우 기본 **XGBoost** 컨테이너를 사용합니다. 그런 다음, 사용할 컨테이너와 역할을 지정하여 추정기를 구성합니다. 이러한 구성을 완료하고 나면 하이퍼파라미터를 선택할 수 있습니다. 아래 코드에 나와 있는 기본값을 사용할 수도 있고 데이터 준비 중에 확인된 사항을 기반으로 코드를 편집할 수도 있습니다.

훈련 작업을 실행하려면 입력을 **data_inputs** 변수로 설정하고 **run_name** 및 **experiment_name**의 구성을 설정하여 **fit()** 을 호출합니다.

In [ ]:

from sagemaker import image_uris
#train-model
# Retrieve the container image
container = sagemaker.image_uris.retrieve(
    region=boto3.Session().region_name, 
    framework="xgboost", 
    version="1.5-1"
)

# Set the hyperparameters
eta=0.2
gamma=4
max_depth=5
min_child_weight=6
num_round=800
objective='binary:logistic'
subsample=0.8

hyperparameters = {
    "eta":eta,
    "gamma":gamma,
    "max_depth":max_depth,
    "min_child_weight":min_child_weight,
    "num_round":num_round,
    "objective":objective,
    "subsample":subsample
}

# Set up the estimator
xgb = sagemaker.estimator.Estimator(
    container,
    role,    
    instance_count=1, 
    instance_type="ml.m5.4xlarge",
    output_path="s3://{}/{}/output".format(bucket, prefix),
    sagemaker_session=sagemaker_session,
    max_run=1800,
    hyperparameters=hyperparameters,
    tags = run_tags
)

with Run(
    experiment_name=capstone_experiment_name,
    run_name=capstone_run_name,
    sagemaker_session=sagemaker_session,
) as run:
    run.log_parameter("eta", eta)
    run.log_parameter("gamma", gamma)
    run.log_parameter("max_depth", max_depth)
    run.log_parameter("min_child_weight", min_child_weight)
    run.log_parameter("objective", objective)
    run.log_parameter("subsample", subsample)
    run.log_parameter("num_round", num_round)

# Train the model associating the training run with the current "experiment"
    xgb.fit(
        inputs = data_inputs
    )        

이 실습을 계속 진행하려면 <a href="#task3-3-continue" target="_self">과제 3.3</a>으로 돌아갑니다.

<a name="task3-3-2" id="task3-3-2"></a>

### 부록: SageMaker Debugger를 사용하도록 설정하여 훈련 작업을 구성 및 실행한 후 보고서 분석(과제 3.3)

SageMaker Debugger에서는 하이퍼파라미터 튜닝 정보를 빠르게 파악할 수 있는 추가 보고서를 찾을 수 있으므로 하이퍼파라미터 범위를 지정해 추가 훈련 작업 실행을 시작할 때 시간을 절약할 수 있습니다. 디버거를 사용하도록 설정하려면 **DebuggerHookConfig** 및 **rules**를 구성합니다.

In [ ]:
#enable-debugger
# Retrieve the container image
container = sagemaker.image_uris.retrieve(
    region=boto3.Session().region_name, 
    framework="xgboost", 
    version="1.5-1"
)

# Set the hyperparameters
eta=0.2
gamma=4
max_depth=5
min_child_weight=6
num_round=300
objective='binary:logistic'
subsample=0.7
        
hyperparameters = {
        "eta":eta,
        "gamma":gamma,
        "max_depth":max_depth,
        "min_child_weight":min_child_weight,
        "num_round":num_round,
        "objective":objective,
        "subsample":subsample
}

# Set up the estimator
xgb = sagemaker.estimator.Estimator(
    container,
    role, 
    base_job_name=base_job_name,
    instance_count=1, 
    instance_type="ml.m5.4xlarge",
    output_path="s3://{}/{}/output".format(bucket, prefix),
    sagemaker_session=sagemaker_session,
    max_run=1800,
    hyperparameters=hyperparameters,
    tags = run_tags,

    #Set the Debugger Hook Config
    debugger_hook_config=DebuggerHookConfig(
        s3_output_path=bucket_path,  # Required
        collection_configs=[
            CollectionConfig(name="metrics", parameters={"save_interval": str(save_interval)}),
            CollectionConfig(name="feature_importance", parameters={"save_interval": str(save_interval)},),
            CollectionConfig(name="full_shap", parameters={"save_interval": str(save_interval)}),
            CollectionConfig(name="average_shap", parameters={"save_interval": str(save_interval)}),
        ],
        ),
        #Set the Debugger Profiler Configuration
        profiler_config = ProfilerConfig(
            system_monitor_interval_millis=500,
            framework_profile_params=FrameworkProfile()
    ),
        #Configure the Debugger Rule Object
        rules = [
            ProfilerRule.sagemaker(rule_configs.ProfilerReport()),
            Rule.sagemaker(rule_configs.create_xgboost_report()),  
            Rule.sagemaker(rule_configs.overfit()),
            Rule.sagemaker(rule_configs.overtraining()),
            Rule.sagemaker(rule_configs.loss_not_decreasing(),
                rule_parameters={
                    "collection_names": "metrics",
                    "num_steps": str(save_interval * 2),
                }
            )
    ]
)
with Run(
    experiment_name=capstone_experiment_name,
    run_name=capstone_run_name,
    sagemaker_session=sagemaker_session,
) as run:
    run.log_parameter("eta", eta)
    run.log_parameter("gamma", gamma)
    run.log_parameter("max_depth", max_depth)
    run.log_parameter("min_child_weight", min_child_weight)
    run.log_parameter("objective", objective)
    run.log_parameter("subsample", subsample)
    run.log_parameter("num_round", num_round)
# Train the model
xgb.fit(
    inputs = data_inputs
) 

이 실습을 계속 진행하려면 <a href="#task3-3-continue" target="_self">과제 3.3</a>으로 돌아갑니다.

<a name="task3-4" id="task3-4"></a>

### 부록: 훈련 하이퍼파라미터 범위 구성(과제 3.4)

모델을 하나 이상 훈련했으므로 데이터 처리 작업과 SageMaker Debugger에서 파악한 정보를 사용하여 하이퍼파라미터용으로 선택해야 하는 범위를 결정할 수 있습니다. 다음 하이퍼파라미터 범위를 편집하고 튜닝 작업을 실행합니다.

In [ ]:
#tune-model
# Setup the hyperparameter ranges
hyperparameter_ranges = {
    "alpha": ContinuousParameter(0, 2),
    "eta": ContinuousParameter(0, 1),
    "max_depth": IntegerParameter(1, 10),
    "min_child_weight": ContinuousParameter(1, 10),
    "num_round": IntegerParameter(100, 1000)
}

# Define the target metric and the objective type (max/min)
objective_metric_name = "validation:auc"
objective_type="Maximize"

# Define the HyperparameterTuner
tuner = HyperparameterTuner(
    estimator = xgb,
    objective_metric_name = objective_metric_name,
    hyperparameter_ranges = hyperparameter_ranges,
    objective_type = objective_type,
    max_jobs=12,
    max_parallel_jobs=4,
    early_stopping_type="Auto"
)

# Tune the model
tuner.fit(
    inputs = data_inputs
)

이 실습을 계속 진행하려면 <a href="#task3-4-continue" target="_self">과제 3.4</a>로 돌아갑니다.

<a name="task4-1" id="task4-1"></a>

### 부록: 모델 생성(과제 4.1)

**model_name**, **role** 및 **container_def**를 정의하여 XGBoost 모델을 생성하고 이름을 **create_model**로 지정합니다.

In [ ]:
#model-configurations
model_name = "capstone-clarify-model"
model = xgb.create_model(name=model_name)
container_def = model.prepare_container_def()
sagemaker_session.create_model(model_name, role, container_def)

이 실습을 계속 진행하려면 <a href="#task4-1-continue" target="_self">과제 4.1</a>로 돌아갑니다.

<a name="task4-2" id="task4-2"></a>

### 부록: SageMaker Clarify 모델 구성 생성(과제 4.2)

**SageMakerClarifyProcessor**를 사용하여 SageMaker Clarify 모델 구성을 생성합니다. **instance_count** 및 **instance_type**을 설정합니다. Capstone을 시작할 때 생성했던 **role** 및 **session**을 사용합니다.

In [ ]:
#define-clarify-processor
clarify_processor = clarify.SageMakerClarifyProcessor(
    role=role, 
    instance_count=1, 
    instance_type="ml.m5.xlarge", 
    sagemaker_session=sagemaker_session
)

이 실습을 계속 진행하려면 <a href="#task4-2-continue" target="_self">과제 4.2</a>로 돌아갑니다.

<a name="task4-3" id="task4-3"></a>

### 부록: SageMaker Clarify 편향 구성 생성(과제 4.3)

SageMaker Clarify 편향 구성을 생성하려면 데이터의 출력 경로를 선택하고 **label**, **headers** 및 **dataset_type**과 함께 훈련 작업에서 입력 경로를 설정합니다.

그런 다음, **ModelConfig** 및 **ModelPredictedLabelConfig**를 생성합니다.

마지막으로 SageMaker Clarify가 관찰할 필드를 사용하여 **BiasConfig**를 구성합니다. 최초 확인 결과를 토대로 살펴보려는 필드를 추가하고 불필요한 필드는 제거할 수 있습니다.

In [ ]:
#define-data-config
bias_report_output_path = "s3://{}/{}/clarify-bias".format(bucket, prefix)
bias_data_config = clarify.DataConfig(
    s3_data_input_path=train_path,
    s3_output_path=bias_report_output_path,
    label="fraud",
    headers=train_data.columns.to_list(),
    dataset_type="text/csv",
)


In [ ]:
#define-model-config
model_config = clarify.ModelConfig(
    model_name=model_name,
    instance_type="ml.m5.xlarge",
    instance_count=1,
    accept_type="text/csv",
    content_type="text/csv",
)

In [ ]:
#define-label-config
predictions_config = clarify.ModelPredictedLabelConfig(probability_threshold=0.8)

In [ ]:
#define-bias-config
bias_config = clarify.BiasConfig(
    label_values_or_threshold=[1], facet_name="customer_gender_female", facet_values_or_threshold=[0], group_name="customer_age"
)

이 실습을 계속 진행하려면 <a href="#task4-3-continue" target="_self">과제 4.3</a>으로 돌아갑니다.

<a name="task4-4" id="task4-4"></a>

### 부록: SageMaker Clarify를 사용하여 편향, 데이터 및 모델 보고서 실행(과제 4.4)

SageMaker Clarify 작업을 구성했으므로 이제 **run_bias**를 호출하여 해당 작업을 실행합니다.

In [ ]:
#run-bias-report
clarify_processor.run_bias(
    data_config=bias_data_config,
    bias_config=bias_config,
    model_config=model_config,
    model_predicted_label_config=predictions_config,
    pre_training_methods="all",
    post_training_methods="all",
)

이 실습을 계속 진행하려면 <a href="#task4-4-continue" target="_self">과제 4.4</a>로 돌아갑니다.

<a name="task4-5" id="task4-5"></a>

### 부록: 불균형 항목 제거(과제 4.5)

이 예제에서는 **customer_gender_female**을 업샘플링하여 데이터 집합의 편향을 줄입니다. 편향이 포함된 기타 특성이 확인되면 해당 특성의 불균형 항목도 제거할 수 있습니다. 아래 코드에서 **random_state**는 `42`로 설정되어 있지만 이 값은 변경할 수 있습니다.

In [ ]:
#display-summary
gender = train_data["customer_gender_female"]
gender.value_counts()

In [ ]:
#remove-imbalance
sm = SMOTE(random_state=42)
train_data_upsampled, gender_res = sm.fit_resample(train_data, gender)
train_data_upsampled["customer_gender_female"].value_counts()

이 실습을 계속 진행하려면 <a href="#task4-5-continue" target="_self">과제 4.5</a>로 돌아갑니다.

<a name="task4-6" id="task4-6"></a>

### 부록: 모델 재훈련(과제 4.6)

불균형 항목을 찾아냈으며 새 훈련 데이터 집합을 생성했습니다. 해당 데이터 집합을 사용하여 파일을 재훈련합니다. 이렇게 하려면 새 파일을 업로드하고 새 추정기를 생성합니다. 그런 다음 **fit()** 을 사용하여 데이터를 재훈련합니다. 아래 코드에는 여러 하이퍼파라미터가 샘플로 포함되어 있습니다. 재훈련 중에 이러한 하이퍼파라미터를 필요한 대로 추가, 제거 또는 조정하여 최적 모델을 찾을 수 있습니다.

In [ ]:
#upload-upsampled-csv
train_data_upsampled.to_csv("data/upsampled_train.csv", index=False, header=False)
retrain_path = S3Uploader.upload("data/upsampled_train.csv", "s3://{}/{}".format(bucket, prefix))
retrain_input = TrainingInput(retrain_path, content_type="text/csv")

retrain_data_inputs = {
    "train": retrain_input,
    "validation": validation_input
}

In [ ]:
#create-estimator
hyperparameters= {
    "max_depth": "5",
    "eta": "0.2",
    "gamma": "4",
    "min_child_weight": "6",
    "subsample": "0.7",
    "objective": "binary:logistic",
    "num_round": "300",
}

xgb_retrained = sagemaker.estimator.Estimator(
    container,
    role, 
    instance_count=1, 
    instance_type="ml.m5.xlarge",
    output_path="s3://{}/{}/output".format(bucket, prefix),
    sagemaker_session=sagemaker_session,
    hyperparameters=hyperparameters
)

In [ ]:
#retrain-upsampled-data
xgb_retrained.fit(
    inputs = retrain_data_inputs
) 

이 실습을 계속 진행하려면 <a href="#task4-6-continue" target="_self">과제 4.6</a>으로 돌아갑니다.

<a name="task5-1" id="task5-1"></a>

### 부록: 배치 변환 작업 생성(과제 5.1)

모델 추정기에서 **transformer**를 사용하여 배치 변환 작업을 생성합니다. 처리 효율성을 높일 수 있도록 전략을 **MultiRecord**로 설정합니다. 그런 다음 **test_path**를 전달하고 추론이 실행될 때까지 기다립니다.

In [ ]:
#create-batch-transformer
# Use the retrained model if it exists, otherwise, use the original model
try:
    model = xgb_retrained
except NameError:
    model = xgb

# Create the transformer
transformer = model.transformer(
    instance_count=1,
    instance_type="ml.m4.xlarge",
    strategy="MultiRecord",
    assemble_with="Line",
    accept="text/csv",
    output_path="s3://{}/{}/batch-transform/".format(bucket, prefix)
)

In [ ]:
#run-batch-transform-job
test_data_batch = test_data.drop("fraud", axis=1)
test_data_batch.to_csv("test_data_batch.csv", index=False, header=False)
test_path_batch = S3Uploader.upload("test_data_batch.csv", "s3://{}/{}".format(bucket, prefix))

transformer.transform(test_path_batch, content_type="text/csv", split_type="Line", join_source="Input")
transformer.wait()

이 실습을 계속 진행하려면 <a href="#task5-1-continue" target="_self">과제 5.1</a>로 돌아갑니다.

<a name="task5-2" id="task5-2"></a>

### 부록: 배치 변환 작업에서 예측 및 정확도 데이터 확인(과제 5.2)

배치 변환 작업이 완료되면 Amazon S3에 저장되어 있는 예측 데이터를 확인합니다. **transformer**에서 설정한 출력 경로를 참조하여 데이터를 샘플링할 수 있습니다.

이 출력에서는 각 레코드의 끝에 **fraud** 예측이 추가됩니다.

In [ ]:
!aws s3 cp --recursive $transformer.output_path ./
test_data = pd.read_csv("test_data_batch.csv.out")
test_data.head()

이 실습을 계속 진행하려면 <a href="#task5-2-continue" target="_self">과제 5.2</a>로 돌아갑니다.

<a name="task5-3" id="task5-3"></a>

### 부록: SageMaker Studio에서 SageMaker 인스턴스 정리(과제 5.3)

SageMaker Studio에서 모델을 개발할 때는 정리할 인스턴스가 있는지 주기적으로 확인해야 합니다. 정리할 인스턴스가 있다면 SageMaker Studio에서 해당 인스턴스를 종료할 수 있습니다.

1. 왼쪽 메뉴 모음에서 **Running Terminals and Kernels** 아이콘(가운데에 사각형이 있는 원 모양 아이콘)을 선택합니다.

1. 아직 열려 있는 인스턴스가 있다면 각 인스턴스 유형 오른쪽에서 **Shut down** 아이콘을 선택합니다. 

각 인스턴스에서 실행 중인 애플리케이션을 표시하여 닫으려는 애플리케이션을 확인할 수 있습니다.

1. 팝업 창이 나타나면 **Shut down all**을 선택합니다.

1. 목록에 인스턴스가 더 이상 나타나지 않을 때까지 **Refresh List** 아이콘을 주기적으로 선택합니다. 인스턴스가 종료되려면 2-5분 정도 걸릴 수 있습니다.

**capstone_ko_kr.ipynb** 노트북이 사용 중인 인스턴스는 종료하지 않아도 됩니다.

이 실습을 계속 진행하려면 <a href="#task5-3-continue" target="_self">과제 5.3</a>으로 돌아갑니다.

<a name="task6-1" id="task6-1"></a>

### 부록: 파이프라인 구성(과제 6.1)

파이프라인을 생성하려면 파이프라인 프로세스의 각 단계를 정의한 후 실행합니다.

이 예제에서는 다음 단계를 생성합니다.
- **AutoModelProcess**: .csv 파일을 끌어온 후 훈련, 검정, 검증 데이터 집합으로 분할하는 **Processing** 단계입니다.
- **AutoHyperParameterTuning**: 여러 하이퍼파라미터를 사용하여 모델을 튜닝하는 **Tuning** 단계입니다.
- **AutoEvalBestModel**: 최적 모델을 설명하기 위한 평가 보고서를 생성하는 **Processing** 단계입니다.
- **CheckAUCScoreAutoEvaluation**: 평가 지표에 따라 모델을 평가하는 **Condition** 단계입니다. 
- **AutoCreateModel**: 모델을 생성하는 **Model** 단계입니다.
- **RegisterAutoModel-RegisterModel**: 모델을 등록하는 **RegisterModel** 단계입니다.
- **AutoModelConfigFile**: 편향 보고서를 생성하는 **Processing** 단계입니다.
- **AutoTransform**: 배치 변환 작업을 실행하는 **Transform** 단계입니다.
- **ClarifyProcessingStep**: SageMaker Clarify 작업을 실행하는 **Processing** 단계입니다.

파이프라인을 빌드하는 과정에서 문제가 발생하면 다음 코드를 사용자 지정할 수도 있으며 또는 이 코드를 기준으로 하여 파이프라인을 직접 빌드할 수도 있습니다.

In [ ]:
#run-pipeline
# Set the variables
model_name = "Auto-model"
sklearn_processor_version="0.23-1"
model_package_group_name="AutoModelPackageGroup"
pipeline_name= "AutoModelSMPipeline"
clarify_image = sagemaker.image_uris.retrieve(framework='sklearn',version=sklearn_processor_version,region=region)

# Upload files to the default S3 bucket
s3_client.put_object(Bucket=bucket,Key='data/')
s3_client.put_object(Bucket=bucket,Key='input/code/')
s3_client.upload_file(Filename="data/batch_data.csv", Bucket=bucket, Key="data/batch_data.csv")  #If you edit this, make sure to also edit the headers listed in generate_config to match your column names.
s3_client.upload_file(Filename="data/claims_customer.csv", Bucket=bucket, Key="data/claims_customer.csv")  #If you edit this, make sure to also edit the headers listed in generate_config to match your column names.
s3_client.upload_file(Filename="pipelines/evaluate.py", Bucket=bucket, Key="input/code/evaluate.py")
s3_client.upload_file(Filename="pipelines/generate_config.py", Bucket=bucket, Key="input/code/generate_config.py")
s3_client.upload_file(Filename="pipelines/preprocess.py", Bucket=bucket, Key="input/code/preprocess.py")

# Configure important settings. Change the input_data if you want to
# use a file other than the claims_customer.csv and batch_data.csv files.
processing_instance_count = ParameterInteger(
    name="ProcessingInstanceCount",
    default_value=1
)
processing_instance_type = ParameterString(
        name="ProcessingInstanceType",
        default_value="ml.m5.xlarge"
)
training_instance_type = ParameterString(
        name="TrainingInstanceType",
        default_value="ml.m5.xlarge"
)
input_data = ParameterString(
        name="InputData",
        default_value="s3://{}/data/claims_customer.csv".format(bucket), 
)
batch_data = ParameterString(
        name="BatchData",
        default_value="s3://{}/data/batch_data.csv".format(bucket),
)

# Run a scikit-learn script to do data processing on SageMaker using 
# using the SKLearnProcessor class
sklearn_processor = SKLearnProcessor(
        framework_version=sklearn_processor_version,
        instance_type=processing_instance_type.default_value, 
        instance_count=processing_instance_count,
        sagemaker_session=sagemaker_session,
        role=role,
)

# Configure the processing step to pull in the input_data
step_process = ProcessingStep(
        name="AutoModelProcess",
        processor=sklearn_processor,
        outputs=[
            ProcessingOutput(output_name="train", source="/opt/ml/processing/train",\
                             destination=f"s3://{bucket}/output/train" ),
            ProcessingOutput(output_name="validation", source="/opt/ml/processing/validation",\
                            destination=f"s3://{bucket}/output/validation"),
            ProcessingOutput(output_name="test", source="/opt/ml/processing/test",\
                            destination=f"s3://{bucket}/output/test"),
            ProcessingOutput(output_name="batch", source="/opt/ml/processing/batch",\
                            destination=f"s3://{bucket}/data/batch"),
            ProcessingOutput(output_name="baseline", source="/opt/ml/processing/baseline",\
                            destination=f"s3://{bucket}/input/baseline")
        ],
        code=f"s3://{bucket}/input/code/preprocess.py",
        job_arguments=["--input-data", input_data],
)

# Set up the model path, image uri, and hyperparameters for the estimator
model_path = f"s3://{bucket}/output"
image_uri = sagemaker.image_uris.retrieve(
    framework="xgboost",
    region=region,
    version="1.5-1",
    py_version="py3",
    instance_type=training_instance_type.default_value,
)

fixed_hyperparameters = {
    "eval_metric":"auc",
    "objective":"binary:logistic",
    "num_round":"100",
    "rate_drop":"0.3",
    "tweedie_variance_power":"1.4"
}

xgb_train = Estimator(
    image_uri=image_uri,
    instance_type=training_instance_type,
    instance_count=1,
    hyperparameters=fixed_hyperparameters,
    output_path=model_path,
    base_job_name=f"auto-train",
    sagemaker_session=sagemaker_session,
    role=role,
)

# Set the hyperparameter ranges for the tuning step and configure the tuning step
hyperparameter_ranges = {
    "eta": ContinuousParameter(0, 1),
    "min_child_weight": ContinuousParameter(1, 10),
    "alpha": ContinuousParameter(0, 2),
    "max_depth": IntegerParameter(1, 10),
}
objective_metric_name = "validation:auc"

step_tuning = TuningStep(
    name = "AutoHyperParameterTuning",
    tuner = HyperparameterTuner(xgb_train, objective_metric_name, hyperparameter_ranges, max_jobs=2, max_parallel_jobs=2),
    inputs={
        "train": TrainingInput(
            s3_data=step_process.properties.ProcessingOutputConfig.Outputs[
                "train"
            ].S3Output.S3Uri,
            content_type="text/csv",
        ),
        "validation": TrainingInput(
            s3_data=step_process.properties.ProcessingOutputConfig.Outputs[
                "validation"
            ].S3Output.S3Uri,
            content_type="text/csv",
        ),
    },
)

# Configure the processing step for evaluation
script_eval = ScriptProcessor(
    image_uri=image_uri,
    command=["python3"],
    instance_type=processing_instance_type,
    instance_count=1,
    base_job_name="script-auto-eval",
    role=role,
    sagemaker_session=sagemaker_session,
)

evaluation_report = PropertyFile(
    name="AutoEvaluationReport",
    output_name="evaluation",
    path="evaluation.json",
)

step_eval = ProcessingStep(
    name="AutoEvalBestModel",
    processor=script_eval,
    inputs=[
        ProcessingInput(
            source=step_tuning.get_top_model_s3_uri(top_k=0,s3_bucket=bucket,prefix="output"),
            destination="/opt/ml/processing/model"
        ),
        ProcessingInput(
            source=step_process.properties.ProcessingOutputConfig.Outputs[
                "test"
            ].S3Output.S3Uri,
            destination="/opt/ml/processing/test"
        )
    ],
    outputs=[
        ProcessingOutput(output_name="evaluation", source="/opt/ml/processing/evaluation",\
                            destination=f"s3://{bucket}/output/evaluation"),
    ],
    code=f"s3://{bucket}/input/code/evaluate.py",
    property_files=[evaluation_report],
)

# Configure model creation
model = Model(
    image_uri=image_uri,        
    model_data=step_tuning.get_top_model_s3_uri(top_k=0,s3_bucket=bucket,prefix="output"),
    name=model_name,
    sagemaker_session=sagemaker_session,
    role=role,
)

inputs = CreateModelInput(
    instance_type="ml.m5.large",
    accelerator_type="ml.inf1.xlarge",
)

step_create_model = CreateModelStep(
    name="AutoCreateModel",
    model=model,
    inputs=inputs,
)

script_processor = ScriptProcessor(
    command=['python3'],
    image_uri=clarify_image,
    role=role,
    instance_count=1,
    instance_type=processing_instance_type,
    sagemaker_session=sagemaker_session,
)

bias_report_output_path = f"s3://{bucket}/clarify-output/bias"
clarify_instance_type = 'ml.m5.xlarge'
step_config_file = ProcessingStep(
    name="AutoModelConfigFile",
    processor=script_processor,
    code=f"s3://{bucket}/input/code/generate_config.py",
    job_arguments=["--modelname",step_create_model.properties.ModelName,"--bias-report-output-path",bias_report_output_path,"--clarify-instance-type",clarify_instance_type,\
                  "--default-bucket",bucket,"--num-baseline-samples","50","--instance-count","1"],
    depends_on= [step_create_model.name]
)

# Configure the step to perform a batch transform job
transformer = Transformer(
    model_name=step_create_model.properties.ModelName,
    instance_type="ml.m5.xlarge",
    instance_count=1,
    assemble_with="Line",
    accept="text/csv",    
    output_path=f"s3://{bucket}/AutoTransform"
)

step_transform = TransformStep(
    name="AutoTransform",
    transformer=transformer,
    inputs=TransformInput(data=batch_data,content_type="text/csv",join_source="Input",split_type="Line")
)

# Configure the SageMaker Clarify processing step
analysis_config_path = f"s3://{bucket}/clarify-output/bias/analysis_config.json"

data_config = sagemaker.clarify.DataConfig(
    s3_data_input_path=f's3://{bucket}/output/train/train.csv', 
    s3_output_path=bias_report_output_path,
    label=0,
    headers=list(pd.read_csv("./data/claims_customer.csv", index_col=None).columns), #If you edit this, make sure to also edit the headers listed in generate_config to match your column names.
    dataset_type="text/csv",
)

clarify_processor = sagemaker.clarify.SageMakerClarifyProcessor(
    role=role,
    instance_count=1,
    instance_type=clarify_instance_type,
    sagemaker_session=sagemaker_session,
)

config_input = ProcessingInput(
    input_name="analysis_config",
    source=analysis_config_path,
    destination="/opt/ml/processing/input/analysis_config",
    s3_data_type="S3Prefix",
    s3_input_mode="File",
    s3_compression_type="None",
)

data_input = ProcessingInput(
    input_name="dataset",
    source=data_config.s3_data_input_path,
    destination="/opt/ml/processing/input/data",
    s3_data_type="S3Prefix",
    s3_input_mode="File",
    s3_data_distribution_type=data_config.s3_data_distribution_type,
    s3_compression_type=data_config.s3_compression_type,
)

result_output = ProcessingOutput(
    source="/opt/ml/processing/output",
    destination=data_config.s3_output_path,
    output_name="analysis_result",
    s3_upload_mode="EndOfJob",
)

step_clarify = ProcessingStep(
    name="ClarifyProcessingStep",
    processor=clarify_processor,
    inputs= [data_input, config_input],
    outputs=[result_output],
    depends_on = [step_config_file.name]
)

# Configure the model registration step
model_statistics = MetricsSource(
    s3_uri="s3://{}/output/evaluation/evaluation.json".format(bucket),
    content_type="application/json"
)
explainability = MetricsSource(
    s3_uri="s3://{}/clarify-output/bias/analysis.json".format(bucket),
    content_type="application/json"
)

bias = MetricsSource(
    s3_uri="s3://{}/clarify-output/bias/analysis.json".format(bucket),
    content_type="application/json"
) 

model_metrics = ModelMetrics(
    model_statistics=model_statistics,
    explainability=explainability,
    bias=bias
)

step_register = RegisterModel(
    name="RegisterAutoModel",
    estimator=xgb_train,
    model_data=step_tuning.get_top_model_s3_uri(top_k=0,s3_bucket=bucket,prefix="output"),
    content_types=["text/csv"],
    response_types=["text/csv"],
    inference_instances=["ml.t2.medium", "ml.m5.large"],
    transform_instances=["ml.m5.large"],
    model_package_group_name=model_package_group_name,
    model_metrics=model_metrics,
)

# Create the model evaluation step
cond_lte = ConditionGreaterThan(
    left=JsonGet(
        step=step_eval,
        property_file=evaluation_report,
        json_path="binary_classification_metrics.auc.value"
    ),
    right=0.75,
)

step_cond = ConditionStep(
    name="CheckAUCScoreAutoEvaluation",
    conditions=[cond_lte],
    if_steps=[step_create_model,step_config_file,step_transform,step_clarify,step_register],
    else_steps=[],
)

# Define the pipeline
def get_pipeline(
    region,
    role=None,
    default_bucket=None,
    model_package_group_name="AutoModelPackageGroup",
    pipeline_name="AutoModelPipeline",
    base_prefix = None,
    custom_image_uri = None,
    sklearn_processor_version=None
    ):
    """Gets a SageMaker ML Pipeline instance working with auto data.
    Args:
        region: AWS region to create and run the pipeline.
        role: IAM role to create and run steps and pipeline.
        default_bucket: the bucket to use for storing the artifacts
    Returns:
        an instance of a pipeline
    """

    # pipeline instance
    pipeline = Pipeline(
        name=pipeline_name,
        parameters=[
            processing_instance_type,
            processing_instance_count,
            training_instance_type,
            input_data,
            batch_data,
        ],
        steps=[step_process,step_tuning,step_eval,step_cond],
        sagemaker_session=sagemaker_session
    )
    return pipeline


# Create the pipeline
pipeline = get_pipeline(
    region = region,
    role=role,
    default_bucket=bucket,
    model_package_group_name=model_package_group_name,
    pipeline_name=pipeline_name,
    custom_image_uri=clarify_image,
    sklearn_processor_version=sklearn_processor_version
)

pipeline.upsert(role_arn=role)

# Run the pipeline
RunPipeline = pipeline.start()

이 실습을 계속 진행하려면 <a href="#task6-1-continue" target="_self">과제 6.1</a>로 돌아갑니다.

<a name="task6-2" id="task6-2"></a>

### 부록: 파이프라인 모니터링(과제 6.2)

파이프라인을 생성하고 실행했으므로 이제 해당 파이프라인을 모니터링합니다. SageMaker Studio에서 파이프라인 상태를 확인할 수 있습니다.

삭제하려는 파이프라인은 **delete_pipeline**을 사용하여 제거하면 됩니다.

In [ ]:
#describe-pipeline
RunPipeline.describe()

In [ ]:
#list-pipeline-steps
RunPipeline.list_steps()

In [ ]:
#remove-pipeline
response = sagemaker_session.boto_session.client("sagemaker", region_name=region).delete_pipeline(PipelineName='AutoModelSMPipeline')
print(response)

이 실습을 계속 진행하려면 <a href="#task6-2-continue" target="_self">과제 6.2</a>로 돌아갑니다.